# Install

In [ ]:
https://docs.kalshi.com/api-reference/events/get-event?playground=open

https://docs.kalshi.com/api-reference/market/get-trades?playground=open

https://www.youtube.com/watch?v=E2mgWN4ReqQ&list=PLvzuUVysUFOtFsOuzlyynhz8kyRvLF9YD&index=4

https://docs.kalshi.com/api-reference/exchange/get-user-data-timestamp#get-user-data-timestamp



# Embedding Model to Match Markets

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentanceTransformer
pd.options.display.max_columns = 200

# Models to use dunzhang/stella_en_1.5B_v5, dunzhang/stella_en_400M_v5
model = SentanceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

# Load data
kalshi_markets = pd.read_json("content/all_kalshi_markets.json")
polymarket_markets = pd.read_json("content/all_polymarkets_markets.json")

# create a column for the retrieval
kalshi_markets["bet_description"] = kalshi_markets["title"] + " " + kalshi_markets["subtitle"] + "\n" + kalshi_markets["rules_primary"] + "\nEnd date" + str(kalshi_markets["close_time"])
polymarket_markets["(bet_description)"] = polymarket_markets["question"] + " " + polymarket_markets["description"] + "\nEnd date" + str(polymarket_markets["end_date_iso"])
kalshi_subset = kalshi_markets.dropna(subset=["bet_description"])
polymarket_subset = polymarket_markets.dropna(subset=["bet_description"])
kalshi_subset.drop_duplicates(subset=["event_ticker"], inplace=True)
print("Kalshi markets ", len(kalshi_subset))
print("Polymarket markets ", len(polymarket_subset))




# Fetch Data

### Get Events

In [ ]:
import requests

# r = requests.get("https://trading-api.kalshi.com/trade-api/events")

r = requests.get("https://api.elections.kalshi.com/trade-api/v2/events/")

# querystring = {"with_nested_markets":"true"}

# response = requests.get(url, params=querystring)

response = r.json()

response

In [ ]:
events = response.get("events", [])

for event in events:
    print(f"{event['title']}  —  ({event['category']}, {event['event_ticker']})")

In [ ]:
import requests
import json

# Fetch all open events (including nested markets)
r = requests.get(
    "https://api.elections.kalshi.com/trade-api/v2/events?with_nested_markets=True&status=open&limit=200"
)
response = r.json()

# Get all events safely
events = response.get("events", [])

# Loop through each event and print the full JSON
for event in events:
    print("==== Event ====")
    print(json.dumps(event, indent=2))  # Pretty-print full event JSON

    print("==== Markets ====")
    markets = event.get("markets", [])
    for market in markets:
        print(json.dumps(market, indent=2))  # Pretty-print full market JSON
    print("\n")  # Add spacing between events


### Fetch Markets

In [ ]:
!pip install requests_cache tqdm

In [6]:
from tqdm import tqdm
import requests

cursor = None
max_items = 100
collected = []
limit = 50  # maybe smaller so you can stop cleanly

with tqdm(total=max_items, desc="Fetching markets…") as progress_bar:
    while len(collected) < max_items:
        url = "https://api.elections.kalshi.com/trade-api/v2/events/"
        params = {
            "limit": limit,
            "with_nested_markets": True,
        }
        if cursor:
            params["cursor"] = cursor

        response = requests.get(url, params=params)
        r = response.json()

        batch = r.get("events", [])
        if not batch:
            break  # no more items

        # add up to the number you need
        remaining = max_items - len(collected)
        collected.extend(batch[:remaining])
        progress_bar.update(min(len(batch), remaining))

        # update cursor for next page
        new_cursor = r.get("cursor")
        if not new_cursor or new_cursor == cursor:
            break  # end pagination
        cursor = new_cursor

# now `collected` has up to max_items items


Fetching markets…: 100%|██████████| 100/100 [00:02<00:00, 42.99it/s]


In [8]:
import json
import pandas as pd

parsed_markets = []

for event in events:
    # Each event is presumably a dict with a key "markets" which is a list of market dicts
    markets = event.get("markets", [])
    for m in markets:
        # you might want to merge the event metadata into the market dict
        # but make a copy to avoid modifying original
        market_copy = m.copy()
        market_copy.update({
            k: v for k, v in event.items() if k != "markets"
        })
        parsed_markets.append(market_copy)

# Now write out the parsed_markets to JSON
today = pd.to_datetime("today").strftime("%Y-%m-%d")
kalshi_file = f"/content/data/kalshi_markets_{today}.json"

with open(kalshi_file, "w") as f:
    json.dump(parsed_markets, f, indent=2)

print(f"Wrote {len(parsed_markets)} markets to {kalshi_file}")


Wrote 759455 markets to /content/data/kalshi_markets_2025-11-07.json


In [ ]:
kalshi = pd.read_json(kalshi_file)
kalshi

### Listen over Websockets

In [ ]:
import asyncio
import websockets
import json
import requests

KALSHI_WEBSOCKET = "wss://api.elections.kalshi.com/market_lifecycle_v2"

data = {"email": "youraccount", "password": "yourpass"}

r = requests.post("https://api.elections.kalshi.com/login", json=data)
response = r.json()

print(response)

In [ ]:
headers = {"Authorization": f"Bearer {response['token']}"}

trade_message = '{"id": 1, "cmd": "subscribe", "params": {"channels": ["trade"]}}'

async def process_message(message):
  print(f"Recieved message: {message}")

async def listen_to_server():
  async with websockets.connect(KALSHI_WEBSOCKET, extra_headers=headers) as websocket:
    await websocket.send(trade_message)
    print(f"Sent trade message: {trade_message}")


# Full Implementation

## **Database**

In [ ]:
"""
Cell 1: SQLite Database Setup for Kalshi API Data
This cell creates all the necessary tables for storing Kalshi API data
"""

import sqlite3
from datetime import datetime

# Create or connect to the database
conn = sqlite3.connect('kalshi_data.db')
cursor = conn.cursor()

# Table 1: Exchange Status
cursor.execute('''
CREATE TABLE IF NOT EXISTS exchange_status (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    exchange_active BOOLEAN,
    exchange_estimated_resume_time TEXT,
    trading_active BOOLEAN,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 2: Exchange Announcements
cursor.execute('''
CREATE TABLE IF NOT EXISTS exchange_announcements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    message TEXT,
    delivery_time DATETIME,
    status TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 3: Series Fee Changes
cursor.execute('''
CREATE TABLE IF NOT EXISTS series_fee_changes (
    id TEXT PRIMARY KEY,
    series_ticker TEXT,
    fee_type TEXT,
    fee_multiplier INTEGER,
    scheduled_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 4: Exchange Schedule
cursor.execute('''
CREATE TABLE IF NOT EXISTS exchange_schedule (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    schedule_type TEXT, -- 'standard' or 'maintenance'
    start_time DATETIME,
    end_time DATETIME,
    day_of_week TEXT,
    open_time TEXT,
    close_time TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 5: User Data Timestamp
cursor.execute('''
CREATE TABLE IF NOT EXISTS user_data_timestamp (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    as_of_time DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 6: Portfolio Balance
cursor.execute('''
CREATE TABLE IF NOT EXISTS portfolio_balance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    balance INTEGER,
    portfolio_value INTEGER,
    updated_ts INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 7: Market Positions
cursor.execute('''
CREATE TABLE IF NOT EXISTS market_positions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticker TEXT,
    total_traded INTEGER,
    total_traded_dollars TEXT,
    position INTEGER,
    market_exposure INTEGER,
    market_exposure_dollars TEXT,
    realized_pnl INTEGER,
    realized_pnl_dollars TEXT,
    resting_orders_count INTEGER,
    fees_paid INTEGER,
    fees_paid_dollars TEXT,
    last_updated_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 8: Event Positions
cursor.execute('''
CREATE TABLE IF NOT EXISTS event_positions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_ticker TEXT,
    total_cost INTEGER,
    total_cost_dollars TEXT,
    total_cost_shares INTEGER,
    event_exposure INTEGER,
    event_exposure_dollars TEXT,
    realized_pnl INTEGER,
    realized_pnl_dollars TEXT,
    resting_order_count INTEGER,
    fees_paid INTEGER,
    fees_paid_dollars TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 9: Settlements
cursor.execute('''
CREATE TABLE IF NOT EXISTS settlements (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticker TEXT,
    market_result TEXT,
    yes_count INTEGER,
    yes_total_cost INTEGER,
    no_count INTEGER,
    no_total_cost INTEGER,
    revenue INTEGER,
    settled_time DATETIME,
    fee_cost TEXT,
    value INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 10: Total Resting Order Value
cursor.execute('''
CREATE TABLE IF NOT EXISTS total_resting_order_value (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    total_resting_order_value INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 11: Fills
cursor.execute('''
CREATE TABLE IF NOT EXISTS fills (
    fill_id TEXT PRIMARY KEY,
    trade_id TEXT,
    order_id TEXT,
    client_order_id TEXT,
    ticker TEXT,
    market_ticker TEXT,
    side TEXT,
    action TEXT,
    count INTEGER,
    price INTEGER,
    yes_price INTEGER,
    no_price INTEGER,
    yes_price_fixed TEXT,
    no_price_fixed TEXT,
    is_taker BOOLEAN,
    created_time DATETIME,
    ts INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 12: Order Groups
cursor.execute('''
CREATE TABLE IF NOT EXISTS order_groups (
    id TEXT PRIMARY KEY,
    is_auto_cancel_enabled BOOLEAN,
    contracts_limit INTEGER,
    status TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 13: Order Group Orders (relationship table)
cursor.execute('''
CREATE TABLE IF NOT EXISTS order_group_orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_group_id TEXT,
    order_id TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (order_group_id) REFERENCES order_groups(id)
)
''')

# Table 14: Orders
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT PRIMARY KEY,
    user_id TEXT,
    client_order_id TEXT,
    ticker TEXT,
    side TEXT,
    action TEXT,
    type TEXT,
    status TEXT,
    yes_price INTEGER,
    no_price INTEGER,
    yes_price_dollars TEXT,
    no_price_dollars TEXT,
    fill_count INTEGER,
    remaining_count INTEGER,
    initial_count INTEGER,
    taker_fees INTEGER,
    maker_fees INTEGER,
    taker_fill_cost INTEGER,
    maker_fill_cost INTEGER,
    taker_fill_cost_dollars TEXT,
    maker_fill_cost_dollars TEXT,
    queue_position INTEGER,
    taker_fees_dollars TEXT,
    maker_fees_dollars TEXT,
    expiration_time DATETIME,
    created_time DATETIME,
    last_update_time DATETIME,
    self_trade_prevention_type TEXT,
    order_group_id TEXT,
    cancel_order_on_pause BOOLEAN,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table 15: Queue Positions
cursor.execute('''
CREATE TABLE IF NOT EXISTS queue_positions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id TEXT,
    market_ticker TEXT,
    queue_position INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Commit the changes
conn.commit()
print("Database setup complete! All tables created successfully.")
print(f"Tables created: {cursor.execute('SELECT name FROM sqlite_master WHERE type=\"table\"').fetchall()}")

# Close the connection
conn.close()

In [ ]:
"""
Cell 27: Additional Database Tables Setup
This cell creates additional tables for the new Kalshi API endpoints
"""

import sqlite3
from datetime import datetime

# Create or connect to the database
conn = sqlite3.connect('kalshi_data.db')
cursor = conn.cursor()

# Table for API Keys
cursor.execute('''
CREATE TABLE IF NOT EXISTS api_keys (
    api_key_id TEXT PRIMARY KEY,
    name TEXT,
    private_key TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Series
cursor.execute('''
CREATE TABLE IF NOT EXISTS series (
    ticker TEXT PRIMARY KEY,
    frequency TEXT,
    title TEXT,
    category TEXT,
    tags TEXT,  -- JSON string
    settlement_sources TEXT,  -- JSON string
    contract_url TEXT,
    contract_terms_url TEXT,
    product_metadata TEXT,  -- JSON string
    fee_type TEXT,
    fee_multiplier INTEGER,
    additional_prohibitions TEXT,  -- JSON string
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Markets
cursor.execute('''
CREATE TABLE IF NOT EXISTS markets (
    ticker TEXT PRIMARY KEY,
    event_ticker TEXT,
    market_type TEXT,
    title TEXT,
    subtitle TEXT,
    yes_sub_title TEXT,
    no_sub_title TEXT,
    open_time DATETIME,
    close_time DATETIME,
    expected_expiration_time DATETIME,
    expiration_time DATETIME,
    latest_expiration_time DATETIME,
    settlement_timer_seconds INTEGER,
    status TEXT,
    response_price_units TEXT,
    yes_bid INTEGER,
    yes_ask INTEGER,
    no_bid INTEGER,
    no_ask INTEGER,
    last_price INTEGER,
    volume INTEGER,
    volume_24h INTEGER,
    result TEXT,
    can_close_early BOOLEAN,
    open_interest INTEGER,
    notional_value INTEGER,
    liquidity INTEGER,
    settlement_value INTEGER,
    category TEXT,
    risk_limit_cents INTEGER,
    fee_waiver_expiration_time DATETIME,
    tick_size INTEGER,
    strike_type TEXT,
    floor_strike INTEGER,
    cap_strike INTEGER,
    rules_primary TEXT,
    rules_secondary TEXT,
    mve_collection_ticker TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Events
cursor.execute('''
CREATE TABLE IF NOT EXISTS events (
    event_ticker TEXT PRIMARY KEY,
    series_ticker TEXT,
    sub_title TEXT,
    title TEXT,
    collateral_return_type TEXT,
    mutually_exclusive BOOLEAN,
    category TEXT,
    strike_date DATETIME,
    strike_period TEXT,
    available_on_brokers BOOLEAN,
    product_metadata TEXT,  -- JSON string
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Trades
cursor.execute('''
CREATE TABLE IF NOT EXISTS trades (
    trade_id TEXT PRIMARY KEY,
    ticker TEXT,
    price INTEGER,
    count INTEGER,
    yes_price INTEGER,
    no_price INTEGER,
    taker_side TEXT,
    created_time DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Candlesticks
cursor.execute('''
CREATE TABLE IF NOT EXISTS candlesticks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticker TEXT,
    end_period_ts INTEGER,
    period_interval INTEGER,
    yes_bid_open INTEGER,
    yes_bid_low INTEGER,
    yes_bid_high INTEGER,
    yes_bid_close INTEGER,
    yes_ask_open INTEGER,
    yes_ask_low INTEGER,
    yes_ask_high INTEGER,
    yes_ask_close INTEGER,
    price_open INTEGER,
    price_low INTEGER,
    price_high INTEGER,
    price_close INTEGER,
    price_mean INTEGER,
    price_previous INTEGER,
    volume INTEGER,
    open_interest INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Orderbook
cursor.execute('''
CREATE TABLE IF NOT EXISTS orderbook (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticker TEXT,
    side TEXT,  -- 'yes' or 'no'
    price_level INTEGER,
    quantity INTEGER,
    snapshot_time DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Event Metadata
cursor.execute('''
CREATE TABLE IF NOT EXISTS event_metadata (
    event_ticker TEXT PRIMARY KEY,
    image_url TEXT,
    settlement_sources TEXT,  -- JSON string
    competition TEXT,
    competition_scope TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Milestones
cursor.execute('''
CREATE TABLE IF NOT EXISTS milestones (
    id TEXT PRIMARY KEY,
    category TEXT,
    type TEXT,
    start_date DATETIME,
    end_date DATETIME,
    related_event_tickers TEXT,  -- JSON string
    title TEXT,
    notification_message TEXT,
    source_id TEXT,
    details TEXT,  -- JSON string
    primary_event_tickers TEXT,  -- JSON string
    last_updated_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Live Data
cursor.execute('''
CREATE TABLE IF NOT EXISTS live_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    milestone_id TEXT,
    details TEXT,  -- JSON string
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Forecast History
cursor.execute('''
CREATE TABLE IF NOT EXISTS forecast_history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_ticker TEXT,
    series_ticker TEXT,
    end_period_ts INTEGER,
    period_interval INTEGER,
    percentile INTEGER,
    raw_numerical_forecast REAL,
    numerical_forecast REAL,
    formatted_forecast TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Volume Incentives
cursor.execute('''
CREATE TABLE IF NOT EXISTS volume_incentives (
    id TEXT PRIMARY KEY,
    market_ticker TEXT,
    incentive_type TEXT,
    start_date DATETIME,
    end_date DATETIME,
    period_reward INTEGER,
    paid_out BOOLEAN,
    discount_factor_bps INTEGER,
    target_size INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Tags by Categories
cursor.execute('''
CREATE TABLE IF NOT EXISTS tags_by_categories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    category TEXT,
    tags TEXT,  -- JSON string
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Sports Filters
cursor.execute('''
CREATE TABLE IF NOT EXISTS sports_filters (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    sport TEXT,
    filters TEXT,  -- JSON string
    sport_order INTEGER,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Commit the changes
conn.commit()
print("Additional database tables created successfully!")
print(f"Total tables now: {len(cursor.execute('SELECT name FROM sqlite_master WHERE type=\"table\"').fetchall())}")

# Close the connection
conn.close()

In [ ]:
"""
Cell 50: Additional Database Tables Setup for FCM and Communications
This cell creates additional tables for FCM, RFQ/Quote, and Multivariate Event Collection endpoints
"""

import sqlite3
from datetime import datetime

# Create or connect to the database
conn = sqlite3.connect('kalshi_data.db')
cursor = conn.cursor()

# Table for FCM Orders (similar to orders but for FCM)
cursor.execute('''
CREATE TABLE IF NOT EXISTS fcm_orders (
    order_id TEXT PRIMARY KEY,
    user_id TEXT,
    subtrader_id TEXT,
    client_order_id TEXT,
    ticker TEXT,
    side TEXT,
    action TEXT,
    type TEXT,
    status TEXT,
    yes_price INTEGER,
    no_price INTEGER,
    yes_price_dollars TEXT,
    no_price_dollars TEXT,
    fill_count INTEGER,
    remaining_count INTEGER,
    initial_count INTEGER,
    taker_fees INTEGER,
    maker_fees INTEGER,
    taker_fill_cost INTEGER,
    maker_fill_cost INTEGER,
    taker_fill_cost_dollars TEXT,
    maker_fill_cost_dollars TEXT,
    queue_position INTEGER,
    taker_fees_dollars TEXT,
    maker_fees_dollars TEXT,
    expiration_time DATETIME,
    created_time DATETIME,
    last_update_time DATETIME,
    self_trade_prevention_type TEXT,
    order_group_id TEXT,
    cancel_order_on_pause BOOLEAN,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for FCM Positions
cursor.execute('''
CREATE TABLE IF NOT EXISTS fcm_positions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    subtrader_id TEXT,
    ticker TEXT,
    event_ticker TEXT,
    position_type TEXT,  -- 'market' or 'event'
    total_traded INTEGER,
    total_traded_dollars TEXT,
    position INTEGER,
    market_exposure INTEGER,
    market_exposure_dollars TEXT,
    event_exposure INTEGER,
    event_exposure_dollars TEXT,
    total_cost INTEGER,
    total_cost_dollars TEXT,
    total_cost_shares INTEGER,
    realized_pnl INTEGER,
    realized_pnl_dollars TEXT,
    resting_orders_count INTEGER,
    resting_order_count INTEGER,
    fees_paid INTEGER,
    fees_paid_dollars TEXT,
    last_updated_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Structured Targets
cursor.execute('''
CREATE TABLE IF NOT EXISTS structured_targets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    details TEXT,  -- JSON string
    source_id TEXT,
    last_updated_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Communications ID
cursor.execute('''
CREATE TABLE IF NOT EXISTS communications_ids (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    communications_id TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for RFQs (Request for Quotes)
cursor.execute('''
CREATE TABLE IF NOT EXISTS rfqs (
    id TEXT PRIMARY KEY,
    creator_id TEXT,
    creator_user_id TEXT,
    market_ticker TEXT,
    contracts INTEGER,
    target_cost_centi_cents INTEGER,
    status TEXT,
    created_ts DATETIME,
    mve_collection_ticker TEXT,
    mve_selected_legs TEXT,  -- JSON string
    rest_remainder BOOLEAN,
    cancellation_reason TEXT,
    cancelled_ts DATETIME,
    updated_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Quotes
cursor.execute('''
CREATE TABLE IF NOT EXISTS quotes (
    id TEXT PRIMARY KEY,
    rfq_id TEXT,
    creator_id TEXT,
    creator_user_id TEXT,
    rfq_creator_id TEXT,
    rfq_creator_user_id TEXT,
    market_ticker TEXT,
    contracts INTEGER,
    yes_bid INTEGER,
    no_bid INTEGER,
    created_ts DATETIME,
    updated_ts DATETIME,
    status TEXT,
    accepted_side TEXT,
    accepted_ts DATETIME,
    confirmed_ts DATETIME,
    executed_ts DATETIME,
    cancelled_ts DATETIME,
    expired_ts DATETIME,
    rest_remainder BOOLEAN,
    cancellation_reason TEXT,
    rfq_target_cost_centi_cents INTEGER,
    rfq_creator_order_id TEXT,
    creator_order_id TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Multivariate Event Collections
cursor.execute('''
CREATE TABLE IF NOT EXISTS multivariate_event_collections (
    collection_ticker TEXT PRIMARY KEY,
    series_ticker TEXT,
    title TEXT,
    description TEXT,
    open_date DATETIME,
    close_date DATETIME,
    associated_events TEXT,  -- JSON string
    associated_event_tickers TEXT,  -- JSON string
    is_ordered BOOLEAN,
    is_single_market_per_event BOOLEAN,
    is_all_yes BOOLEAN,
    size_min INTEGER,
    size_max INTEGER,
    functional_description TEXT,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Table for Multivariate Event Collection Lookups
cursor.execute('''
CREATE TABLE IF NOT EXISTS multivariate_lookups (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    collection_ticker TEXT,
    event_ticker TEXT,
    market_ticker TEXT,
    selected_markets TEXT,  -- JSON string
    last_queried_ts DATETIME,
    fetched_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')

# Commit the changes
conn.commit()
print("FCM and Communications database tables created successfully!")
print(f"New tables added for FCM operations, RFQ/Quote system, and Multivariate Event Collections")

# Close the connection
conn.close()

## **Secrets**

In [ ]:
!pip install cryptography requests

In [13]:
import datetime
import base64
import requests
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

# CONFIG: fill these in
API_KEY_ID = "0189969d-8705-4f78-af17-ca93543146b3"   # from Kalshi
PRIVATE_KEY_FILE = "/content/secrets/colabtest_kalshi.txt"

BASE_URL = "https://api.elections.kalshi.com"  # use correct environment, demo/prod
PATH = "/trade-api/v2/api_keys"

def load_private_key(path):
    with open(path, "rb") as f:
        key_data = f.read()
    private_key = serialization.load_pem_private_key(
        key_data,
        password=None,
        backend=default_backend()
    )
    return private_key

def create_signature(private_key, timestamp_str, method, path):
    path_no_query = path.split('?')[0]
    message = (timestamp_str + method + path_no_query).encode("utf-8")
    sig_bytes = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.DIGEST_LENGTH
        ),
        hashes.SHA256()
    )
    return base64.b64encode(sig_bytes).decode("utf-8")

# Load your key
priv_key = load_private_key(PRIVATE_KEY_FILE)

# Prepare request
method = "GET"
timestamp_ms = str(int(datetime.datetime.now().timestamp() * 1000))
signature = create_signature(priv_key, timestamp_ms, method, PATH)

headers = {
    "KALSHI-ACCESS-KEY": API_KEY_ID,
    "KALSHI-ACCESS-TIMESTAMP": timestamp_ms,
    "KALSHI-ACCESS-SIGNATURE": signature
}

url = BASE_URL + PATH

# Send request
response = requests.get(url, headers=headers)
print("Status code:", response.status_code)
print("Response:", response.json())
# print(API_KEY_ID)
# print(timestamp_ms)
# print(signature)


Status code: 200
Response: {'api_keys': [{'api_key_id': '0189969d-8705-4f78-af17-ca93543146b3', 'name': 'colabtest'}]}
0189969d-8705-4f78-af17-ca93543146b3
1762493118628
Yewn6ebBwVeqdh4aO8UgiBsNz9D4iA7snyCau80db+fg6W0Vlz+C2Ejkqg6GIuJ6twyd0Ufuwv6Em/beLFCD3J0fqEKNHXaStwCl82g7AiOPSe0cv8aWmCysOPvi1QCJOH9hxUdVz47F3ba42cD1xjG6kLeppkywiBr2XByavGvR0J/pN30Pec+MXvZ+6lVb27F43huaXXbX64qvo2tOYKM+lc6Og2govfFUR6Pg7z/I5PEQuXqN7YtWA6YeOnckBsY16Apq8biWw18B+JL+0qm/KJVMC/iis00PvZZkmy4Ftt43hOhRXHJx60SoC82iy0FwMHo78nkWn0IGd4/qAw==


## **Exchange**

#### Get Exchange Status

In [ ]:
"""
Cell 2: Get Exchange Status
Retrieves the current status of the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_exchange_status():
    """
    Fetches the current exchange status from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/exchange/status"

    try:
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO exchange_status (exchange_active, exchange_estimated_resume_time, trading_active)
            VALUES (?, ?, ?)
        ''', (
            data.get('exchange_active'),
            data.get('exchange_estimated_resume_time'),
            data.get('trading_active')
        ))

        conn.commit()
        conn.close()

        print("Exchange Status Retrieved:")
        print(f"  Exchange Active: {data.get('exchange_active')}")
        print(f"  Trading Active: {data.get('trading_active')}")
        print(f"  Estimated Resume Time: {data.get('exchange_estimated_resume_time')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching exchange status: {e}")
        return None
    except Exception as e:
        print(f"Error processing exchange status: {e}")
        return None

# Execute the function
exchange_status_data = get_exchange_status()

#### Get Exchange Announcements

In [ ]:
"""
Cell 3: Get Exchange Announcements
Retrieves announcements from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_exchange_announcements():
    """
    Fetches exchange announcements from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/exchange/announcements"

    try:
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        announcements = data.get('announcements', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for announcement in announcements:
            cursor.execute('''
                INSERT INTO exchange_announcements (type, message, delivery_time, status)
                VALUES (?, ?, ?, ?)
            ''', (
                announcement.get('type'),
                announcement.get('message'),
                announcement.get('delivery_time'),
                announcement.get('status')
            ))

        conn.commit()
        conn.close()

        print(f"Exchange Announcements Retrieved: {len(announcements)} announcements")
        for ann in announcements:
            print(f"  - Type: {ann.get('type')}, Status: {ann.get('status')}")
            print(f"    Message: {ann.get('message')[:100]}..." if len(ann.get('message', '')) > 100 else f"    Message: {ann.get('message')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching exchange announcements: {e}")
        return None
    except Exception as e:
        print(f"Error processing exchange announcements: {e}")
        return None

# Execute the function
announcements_data = get_exchange_announcements()

#### Get Series Fee Changes

In [ ]:
"""
Cell 4: Get Series Fee Changes
Retrieves series fee changes from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_series_fee_changes():
    """
    Fetches series fee changes from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/series/fee_changes"

    try:
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        fee_changes = data.get('series_fee_change_arr', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for fee_change in fee_changes:
            # Use REPLACE to handle potential duplicates by id
            cursor.execute('''
                INSERT OR REPLACE INTO series_fee_changes (id, series_ticker, fee_type, fee_multiplier, scheduled_ts)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                fee_change.get('id'),
                fee_change.get('series_ticker'),
                fee_change.get('fee_type'),
                fee_change.get('fee_multiplier'),
                fee_change.get('scheduled_ts')
            ))

        conn.commit()
        conn.close()

        print(f"Series Fee Changes Retrieved: {len(fee_changes)} changes")
        for change in fee_changes:
            print(f"  - Series: {change.get('series_ticker')}, Type: {change.get('fee_type')}")
            print(f"    Multiplier: {change.get('fee_multiplier')}, Scheduled: {change.get('scheduled_ts')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching series fee changes: {e}")
        return None
    except Exception as e:
        print(f"Error processing series fee changes: {e}")
        return None

# Execute the function
fee_changes_data = get_series_fee_changes()

#### Get Exchange Schedule

In [ ]:
"""
Cell 5: Get Exchange Schedule
Retrieves the exchange schedule from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_exchange_schedule():
    """
    Fetches exchange schedule from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/exchange/schedule"

    try:
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        schedule = data.get('schedule', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Process standard hours
        standard_hours = schedule.get('standard_hours', [])
        for period in standard_hours:
            start_time = period.get('start_time')
            end_time = period.get('end_time')

            # Process each day of the week
            days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
            for day in days:
                day_schedule = period.get(day, [])
                for time_slot in day_schedule:
                    cursor.execute('''
                        INSERT INTO exchange_schedule (schedule_type, start_time, end_time, day_of_week, open_time, close_time)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        'standard',
                        start_time,
                        end_time,
                        day,
                        time_slot.get('open_time'),
                        time_slot.get('close_time')
                    ))

        # Process maintenance windows
        maintenance_windows = schedule.get('maintenance_windows', [])
        for window in maintenance_windows:
            cursor.execute('''
                INSERT INTO exchange_schedule (schedule_type, start_time, end_time)
                VALUES (?, ?, ?)
            ''', (
                'maintenance',
                window.get('start_datetime'),
                window.get('end_datetime')
            ))

        conn.commit()
        conn.close()

        print(f"Exchange Schedule Retrieved:")
        print(f"  Standard Hours Periods: {len(standard_hours)}")
        print(f"  Maintenance Windows: {len(maintenance_windows)}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching exchange schedule: {e}")
        return None
    except Exception as e:
        print(f"Error processing exchange schedule: {e}")
        return None

# Execute the function
schedule_data = get_exchange_schedule()

#### Get User Data Timestamp

In [ ]:
"""
Cell 6: Get User Data Timestamp
Retrieves the user data timestamp from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_user_data_timestamp():
    """
    Fetches user data timestamp from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/exchange/user_data_timestamp"

    try:
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO user_data_timestamp (as_of_time)
            VALUES (?)
        ''', (data.get('as_of_time'),))

        conn.commit()
        conn.close()

        print(f"User Data Timestamp Retrieved: {data.get('as_of_time')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching user data timestamp: {e}")
        return None
    except Exception as e:
        print(f"Error processing user data timestamp: {e}")
        return None

# Execute the function
user_data_timestamp = get_user_data_timestamp()

## **Portfolio**

#### Get Balance

In [ ]:
"""
Cell 7: Get Balance
Retrieves the user's balance from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_balance(headers):
    """
    Fetches user balance from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
                 (KALSHI-ACCESS-KEY, KALSHI-ACCESS-SIGNATURE, KALSHI-ACCESS-TIMESTAMP)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/balance"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO portfolio_balance (balance, portfolio_value, updated_ts)
            VALUES (?, ?, ?)
        ''', (
            data.get('balance'),
            data.get('portfolio_value'),
            data.get('updated_ts')
        ))

        conn.commit()
        conn.close()

        print("Portfolio Balance Retrieved:")
        print(f"  Balance: ${data.get('balance', 0) / 100:.2f}")  # Assuming cents
        print(f"  Portfolio Value: ${data.get('portfolio_value', 0) / 100:.2f}")
        print(f"  Updated: {data.get('updated_ts')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching balance: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing balance: {e}")
        return None

# Execute the function
# Note: headers variable should already be defined with authentication details
try:
    balance_data = get_balance(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    balance_data = None

#### Get Positions

In [ ]:
"""
Cell 8: Get Positions
Retrieves the user's positions from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_positions(headers, limit=100, settlement_status="unsettled", count_filter=None,
                  ticker=None, event_ticker=None, cursor=None):
    """
    Fetches user positions from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        limit: Number of results per page (1-1000, default 100)
        settlement_status: 'all', 'unsettled', or 'settled' (default 'unsettled')
        count_filter: Filter positions with non-zero values (comma-separated: position, total_traded, resting_order_count)
        ticker: Filter by market ticker
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/positions"

    # Build query parameters
    params = {"limit": limit, "settlement_status": settlement_status}
    if count_filter:
        params["count_filter"] = count_filter
    if ticker:
        params["ticker"] = ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        # Store market positions
        market_positions = data.get('market_positions', [])
        for position in market_positions:
            cursor_db.execute('''
                INSERT INTO market_positions (
                    ticker, total_traded, total_traded_dollars, position,
                    market_exposure, market_exposure_dollars, realized_pnl,
                    realized_pnl_dollars, resting_orders_count, fees_paid,
                    fees_paid_dollars, last_updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                position.get('ticker'),
                position.get('total_traded'),
                position.get('total_traded_dollars'),
                position.get('position'),
                position.get('market_exposure'),
                position.get('market_exposure_dollars'),
                position.get('realized_pnl'),
                position.get('realized_pnl_dollars'),
                position.get('resting_orders_count'),
                position.get('fees_paid'),
                position.get('fees_paid_dollars'),
                position.get('last_updated_ts')
            ))

        # Store event positions
        event_positions = data.get('event_positions', [])
        for position in event_positions:
            cursor_db.execute('''
                INSERT INTO event_positions (
                    event_ticker, total_cost, total_cost_dollars, total_cost_shares,
                    event_exposure, event_exposure_dollars, realized_pnl,
                    realized_pnl_dollars, resting_order_count, fees_paid, fees_paid_dollars
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                position.get('event_ticker'),
                position.get('total_cost'),
                position.get('total_cost_dollars'),
                position.get('total_cost_shares'),
                position.get('event_exposure'),
                position.get('event_exposure_dollars'),
                position.get('realized_pnl'),
                position.get('realized_pnl_dollars'),
                position.get('resting_order_count'),
                position.get('fees_paid'),
                position.get('fees_paid_dollars')
            ))

        conn.commit()
        conn.close()

        print(f"Positions Retrieved:")
        print(f"  Market Positions: {len(market_positions)}")
        print(f"  Event Positions: {len(event_positions)}")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching positions: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing positions: {e}")
        return None

# Execute the function with default parameters
# Note: headers variable should already be defined with authentication details
try:
    positions_data = get_positions(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    positions_data = None

#### Get Settlements

In [ ]:
"""
Cell 9: Get Settlements
Retrieves the user's settlements from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_settlements(headers, limit=100, cursor=None, ticker=None,
                    event_ticker=None, min_ts=None, max_ts=None):
    """
    Fetches user settlements from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        limit: Number of results per page (1-200, default 100)
        cursor: Pagination cursor from previous response
        ticker: Filter by market ticker
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        min_ts: Filter items after this Unix timestamp
        max_ts: Filter items before this Unix timestamp
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/settlements"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if ticker:
        params["ticker"] = ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if min_ts:
        params["min_ts"] = min_ts
    if max_ts:
        params["max_ts"] = max_ts

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        settlements = data.get('settlements', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for settlement in settlements:
            cursor_db.execute('''
                INSERT INTO settlements (
                    ticker, market_result, yes_count, yes_total_cost,
                    no_count, no_total_cost, revenue, settled_time,
                    fee_cost, value
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                settlement.get('ticker'),
                settlement.get('market_result'),
                settlement.get('yes_count'),
                settlement.get('yes_total_cost'),
                settlement.get('no_count'),
                settlement.get('no_total_cost'),
                settlement.get('revenue'),
                settlement.get('settled_time'),
                settlement.get('fee_cost'),
                settlement.get('value')
            ))

        conn.commit()
        conn.close()

        print(f"Settlements Retrieved: {len(settlements)} settlements")
        for settlement in settlements[:5]:  # Show first 5
            print(f"  - Ticker: {settlement.get('ticker')}, Result: {settlement.get('market_result')}")
            print(f"    Revenue: ${settlement.get('revenue', 0) / 100:.2f}")
        if len(settlements) > 5:
            print(f"  ... and {len(settlements) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching settlements: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing settlements: {e}")
        return None

# Execute the function with default parameters
# Note: headers variable should already be defined with authentication details
try:
    settlements_data = get_settlements(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    settlements_data = None

#### Get Total Resting Order Value

In [ ]:
"""
Cell 10: Get Total Resting Order Value
Retrieves the total resting order value from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_total_resting_order_value(headers):
    """
    Fetches total resting order value from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
                 (KALSHI-ACCESS-KEY, KALSHI-ACCESS-SIGNATURE, KALSHI-ACCESS-TIMESTAMP)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/summary/total_resting_order_value"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO total_resting_order_value (total_resting_order_value)
            VALUES (?)
        ''', (data.get('total_resting_order_value'),))

        conn.commit()
        conn.close()

        value = data.get('total_resting_order_value', 0)
        print(f"Total Resting Order Value Retrieved: ${value / 100:.2f}")  # Assuming cents

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching total resting order value: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing total resting order value: {e}")
        return None

# Execute the function
# Note: headers variable should already be defined with authentication details
try:
    resting_order_value_data = get_total_resting_order_value(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    resting_order_value_data = None

#### Get Fills

In [ ]:
"""
Cell 11: Get Fills
Retrieves the user's fills from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_fills(headers, limit=100, ticker=None, order_id=None,
              min_ts=None, max_ts=None, cursor=None):
    """
    Fetches user fills from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        limit: Number of results per page (1-200, default 100)
        ticker: Filter by market ticker
        order_id: Filter by order ID
        min_ts: Filter items after this Unix timestamp
        max_ts: Filter items before this Unix timestamp
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/fills"

    # Build query parameters
    params = {"limit": limit}
    if ticker:
        params["ticker"] = ticker
    if order_id:
        params["order_id"] = order_id
    if min_ts:
        params["min_ts"] = min_ts
    if max_ts:
        params["max_ts"] = max_ts
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        fills = data.get('fills', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for fill in fills:
            cursor_db.execute('''
                INSERT OR REPLACE INTO fills (
                    fill_id, trade_id, order_id, client_order_id, ticker,
                    market_ticker, side, action, count, price, yes_price,
                    no_price, yes_price_fixed, no_price_fixed, is_taker,
                    created_time, ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                fill.get('fill_id'),
                fill.get('trade_id'),
                fill.get('order_id'),
                fill.get('client_order_id'),
                fill.get('ticker'),
                fill.get('market_ticker'),
                fill.get('side'),
                fill.get('action'),
                fill.get('count'),
                fill.get('price'),
                fill.get('yes_price'),
                fill.get('no_price'),
                fill.get('yes_price_fixed'),
                fill.get('no_price_fixed'),
                fill.get('is_taker'),
                fill.get('created_time'),
                fill.get('ts')
            ))

        conn.commit()
        conn.close()

        print(f"Fills Retrieved: {len(fills)} fills")
        for fill in fills[:5]:  # Show first 5
            print(f"  - Ticker: {fill.get('ticker')}, Side: {fill.get('side')}, Action: {fill.get('action')}")
            print(f"    Count: {fill.get('count')}, Price: {fill.get('price')}")
        if len(fills) > 5:
            print(f"  ... and {len(fills) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching fills: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing fills: {e}")
        return None

# Execute the function with default parameters
# Note: headers variable should already be defined with authentication details
try:
    fills_data = get_fills(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    fills_data = None

#### Get Order Groups

In [ ]:
"""
Cell 12: Get Order Groups
Retrieves the user's order groups from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_order_groups(headers, limit=100, status=None, cursor=None):
    """
    Fetches user order groups from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        limit: Number of results per page (1-200, default 100)
        status: Filter by status
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/order_groups"

    # Build query parameters
    params = {"limit": limit}
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order_groups = data.get('order_groups', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for group in order_groups:
            # Store or update order group
            cursor_db.execute('''
                INSERT OR REPLACE INTO order_groups (id, is_auto_cancel_enabled, status)
                VALUES (?, ?, ?)
            ''', (
                group.get('id'),
                group.get('is_auto_cancel_enabled'),
                status  # Save the status if provided
            ))

        conn.commit()
        conn.close()

        print(f"Order Groups Retrieved: {len(order_groups)} groups")
        for group in order_groups[:5]:  # Show first 5
            print(f"  - ID: {group.get('id')}")
            print(f"    Auto Cancel Enabled: {group.get('is_auto_cancel_enabled')}")
        if len(order_groups) > 5:
            print(f"  ... and {len(order_groups) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching order groups: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order groups: {e}")
        return None

# Execute the function with default parameters
# Note: headers variable should already be defined with authentication details
try:
    order_groups_data = get_order_groups(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    order_groups_data = None

#### Create Order Group

In [ ]:
"""
Cell 13: Create Order Group
Creates a new order group in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def create_order_group(headers, contracts_limit):
    """
    Creates a new order group in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        contracts_limit: Maximum number of contracts that can be matched within this group (must be >= 1)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/order_groups/create"

    # Prepare payload
    payload = {
        "contracts_limit": contracts_limit
    }

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request with authentication headers
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order_group_id = data.get('order_group_id')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO order_groups (id, is_auto_cancel_enabled, contracts_limit)
            VALUES (?, ?, ?)
        ''', (
            order_group_id,
            True,  # Default value, will be updated when fetching details
            contracts_limit
        ))

        conn.commit()
        conn.close()

        print(f"Order Group Created Successfully!")
        print(f"  Order Group ID: {order_group_id}")
        print(f"  Contracts Limit: {contracts_limit}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating order group: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order group creation: {e}")
        return None

# Example usage: Create an order group with a limit of 100 contracts
# Note: headers variable should already be defined with authentication details
try:
    # You can adjust the contracts_limit as needed
    new_order_group = create_order_group(headers, contracts_limit=100)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_order_group = None

#### Get Order Group

In [ ]:
"""
Cell 14: Get Order Group
Retrieves details of a specific order group from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_order_group(headers, order_group_id):
    """
    Fetches a specific order group from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_group_id: The order group ID to retrieve
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/order_groups/{order_group_id}"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Update order group information
        cursor.execute('''
            INSERT OR REPLACE INTO order_groups (id, is_auto_cancel_enabled)
            VALUES (?, ?)
        ''', (
            order_group_id,
            data.get('is_auto_cancel_enabled')
        ))

        # Store order associations
        orders = data.get('orders', [])
        for order_id in orders:
            cursor.execute('''
                INSERT OR REPLACE INTO order_group_orders (order_group_id, order_id)
                VALUES (?, ?)
            ''', (order_group_id, order_id))

        conn.commit()
        conn.close()

        print(f"Order Group Details Retrieved:")
        print(f"  Order Group ID: {order_group_id}")
        print(f"  Auto Cancel Enabled: {data.get('is_auto_cancel_enabled')}")
        print(f"  Number of Orders: {len(orders)}")
        if orders:
            print(f"  Orders: {', '.join(orders[:5])}")
            if len(orders) > 5:
                print(f"    ... and {len(orders) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching order group: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order group: {e}")
        return None

# Example usage: Get a specific order group
# Note: You need to provide a valid order_group_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-group-id' with an actual order group ID
    order_group_id = "your-order-group-id"  # This should be replaced with an actual ID
    order_group_details = get_order_group(headers, order_group_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    order_group_details = None

#### Delete Order Group

In [ ]:
"""
Cell 15: Delete Order Group
Deletes an order group from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def delete_order_group(headers, order_group_id):
    """
    Deletes an order group from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_group_id: The order group ID to delete
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/order_groups/{order_group_id}"

    try:
        # Make the API request with authentication headers
        response = requests.delete(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data (might be empty for successful deletion)
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Remove from database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Delete order group associations
        cursor.execute('DELETE FROM order_group_orders WHERE order_group_id = ?', (order_group_id,))

        # Delete order group
        cursor.execute('DELETE FROM order_groups WHERE id = ?', (order_group_id,))

        conn.commit()
        conn.close()

        print(f"Order Group Deleted Successfully!")
        print(f"  Order Group ID: {order_group_id}")

        return data if data else {"success": True, "deleted_id": order_group_id}

    except requests.exceptions.RequestException as e:
        print(f"Error deleting order group: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order group deletion: {e}")
        return None

# Example usage: Delete a specific order group
# Note: You need to provide a valid order_group_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-group-id' with an actual order group ID to delete
    order_group_id = "your-order-group-id"  # This should be replaced with an actual ID
    deletion_result = delete_order_group(headers, order_group_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    deletion_result = None

#### Reset Order Group

In [ ]:
"""
Cell 16: Reset Order Group
Resets an order group in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def reset_order_group(headers, order_group_id):
    """
    Resets an order group in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_group_id: The order group ID to reset
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/order_groups/{order_group_id}/reset"

    try:
        # Make the API request with authentication headers
        response = requests.put(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Update database to reflect reset
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Clear any order associations for this group (since it's reset)
        cursor.execute('DELETE FROM order_group_orders WHERE order_group_id = ?', (order_group_id,))

        conn.commit()
        conn.close()

        print(f"Order Group Reset Successfully!")
        print(f"  Order Group ID: {order_group_id}")

        return data if data else {"success": True, "reset_id": order_group_id}

    except requests.exceptions.RequestException as e:
        print(f"Error resetting order group: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order group reset: {e}")
        return None

# Example usage: Reset a specific order group
# Note: You need to provide a valid order_group_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-group-id' with an actual order group ID to reset
    order_group_id = "your-order-group-id"  # This should be replaced with an actual ID
    reset_result = reset_order_group(headers, order_group_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    reset_result = None

#### Get Orders

In [ ]:
"""
Cell 17: Get Orders
Retrieves the user's orders from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_orders(headers, limit=100, ticker=None, event_ticker=None,
               min_ts=None, max_ts=None, status=None, cursor=None):
    """
    Fetches user orders from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        limit: Number of results per page (1-200, default 100)
        ticker: Filter by market ticker
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        min_ts: Filter items after this Unix timestamp
        max_ts: Filter items before this Unix timestamp
        status: Filter by status
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/orders"

    # Build query parameters
    params = {"limit": limit}
    if ticker:
        params["ticker"] = ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if min_ts:
        params["min_ts"] = min_ts
    if max_ts:
        params["max_ts"] = max_ts
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        orders = data.get('orders', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for order in orders:
            cursor_db.execute('''
                INSERT OR REPLACE INTO orders (
                    order_id, user_id, client_order_id, ticker, side, action,
                    type, status, yes_price, no_price, yes_price_dollars,
                    no_price_dollars, fill_count, remaining_count, initial_count,
                    taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                    taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                    taker_fees_dollars, maker_fees_dollars, expiration_time,
                    created_time, last_update_time, self_trade_prevention_type,
                    order_group_id, cancel_order_on_pause
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                order.get('order_id'),
                order.get('user_id'),
                order.get('client_order_id'),
                order.get('ticker'),
                order.get('side'),
                order.get('action'),
                order.get('type'),
                order.get('status'),
                order.get('yes_price'),
                order.get('no_price'),
                order.get('yes_price_dollars'),
                order.get('no_price_dollars'),
                order.get('fill_count'),
                order.get('remaining_count'),
                order.get('initial_count'),
                order.get('taker_fees'),
                order.get('maker_fees'),
                order.get('taker_fill_cost'),
                order.get('maker_fill_cost'),
                order.get('taker_fill_cost_dollars'),
                order.get('maker_fill_cost_dollars'),
                order.get('queue_position'),
                order.get('taker_fees_dollars'),
                order.get('maker_fees_dollars'),
                order.get('expiration_time'),
                order.get('created_time'),
                order.get('last_update_time'),
                order.get('self_trade_prevention_type'),
                order.get('order_group_id'),
                order.get('cancel_order_on_pause')
            ))

        conn.commit()
        conn.close()

        print(f"Orders Retrieved: {len(orders)} orders")
        for order in orders[:5]:  # Show first 5
            print(f"  - Order ID: {order.get('order_id')}")
            print(f"    Ticker: {order.get('ticker')}, Side: {order.get('side')}, Action: {order.get('action')}")
            print(f"    Status: {order.get('status')}, Count: {order.get('initial_count')}")
        if len(orders) > 5:
            print(f"  ... and {len(orders) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching orders: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing orders: {e}")
        return None

# Execute the function with default parameters
# Note: headers variable should already be defined with authentication details
try:
    orders_data = get_orders(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    orders_data = None

#### Create Order

In [ ]:
"""
Cell 18: Create Order
Creates a new order in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def create_order(headers, ticker, side, action, count, order_type="limit",
                yes_price=None, no_price=None, yes_price_dollars=None,
                no_price_dollars=None, client_order_id=None, expiration_ts=None,
                time_in_force=None, buy_max_cost=None, post_only=None,
                reduce_only=None, sell_position_floor=None,
                self_trade_prevention_type=None, order_group_id=None,
                cancel_order_on_pause=None):
    """
    Creates a new order in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        ticker: Market ticker (required)
        side: 'yes' or 'no' (required)
        action: 'buy' or 'sell' (required)
        count: Number of contracts (required, must be >= 1)
        order_type: 'limit' or 'market' (default 'limit')
        yes_price: Yes price (1-99)
        no_price: No price (1-99)
        yes_price_dollars: Yes price in fixed-point dollars (e.g., "0.5000")
        no_price_dollars: No price in fixed-point dollars
        client_order_id: Custom order ID
        expiration_ts: Expiration timestamp
        time_in_force: 'fill_or_kill', 'good_till_canceled', 'immediate_or_cancel'
        buy_max_cost: Maximum cost in cents (auto FoK behavior)
        post_only: Boolean for post-only orders
        reduce_only: Boolean for reduce-only orders
        sell_position_floor: Deprecated, use reduce_only
        self_trade_prevention_type: 'taker_at_cross' or 'maker'
        order_group_id: Order group ID
        cancel_order_on_pause: Cancel if trading paused
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/orders"

    # Build payload
    payload = {
        "ticker": ticker,
        "side": side,
        "action": action,
        "count": count,
        "type": order_type
    }

    # Add optional parameters
    if client_order_id:
        payload["client_order_id"] = client_order_id
    if yes_price is not None:
        payload["yes_price"] = yes_price
    if no_price is not None:
        payload["no_price"] = no_price
    if yes_price_dollars:
        payload["yes_price_dollars"] = yes_price_dollars
    if no_price_dollars:
        payload["no_price_dollars"] = no_price_dollars
    if expiration_ts:
        payload["expiration_ts"] = expiration_ts
    if time_in_force:
        payload["time_in_force"] = time_in_force
    if buy_max_cost is not None:
        payload["buy_max_cost"] = buy_max_cost
    if post_only is not None:
        payload["post_only"] = post_only
    if reduce_only is not None:
        payload["reduce_only"] = reduce_only
    if sell_position_floor is not None:
        payload["sell_position_floor"] = sell_position_floor
    if self_trade_prevention_type:
        payload["self_trade_prevention_type"] = self_trade_prevention_type
    if order_group_id:
        payload["order_group_id"] = order_group_id
    if cancel_order_on_pause is not None:
        payload["cancel_order_on_pause"] = cancel_order_on_pause

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order = data.get('order', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        cursor_db.execute('''
            INSERT OR REPLACE INTO orders (
                order_id, user_id, client_order_id, ticker, side, action,
                type, status, yes_price, no_price, yes_price_dollars,
                no_price_dollars, fill_count, remaining_count, initial_count,
                taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                taker_fees_dollars, maker_fees_dollars, expiration_time,
                created_time, last_update_time, self_trade_prevention_type,
                order_group_id, cancel_order_on_pause
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            order.get('order_id'),
            order.get('user_id'),
            order.get('client_order_id'),
            order.get('ticker'),
            order.get('side'),
            order.get('action'),
            order.get('type'),
            order.get('status'),
            order.get('yes_price'),
            order.get('no_price'),
            order.get('yes_price_dollars'),
            order.get('no_price_dollars'),
            order.get('fill_count'),
            order.get('remaining_count'),
            order.get('initial_count'),
            order.get('taker_fees'),
            order.get('maker_fees'),
            order.get('taker_fill_cost'),
            order.get('maker_fill_cost'),
            order.get('taker_fill_cost_dollars'),
            order.get('maker_fill_cost_dollars'),
            order.get('queue_position'),
            order.get('taker_fees_dollars'),
            order.get('maker_fees_dollars'),
            order.get('expiration_time'),
            order.get('created_time'),
            order.get('last_update_time'),
            order.get('self_trade_prevention_type'),
            order.get('order_group_id'),
            order.get('cancel_order_on_pause')
        ))

        conn.commit()
        conn.close()

        print(f"Order Created Successfully!")
        print(f"  Order ID: {order.get('order_id')}")
        print(f"  Ticker: {order.get('ticker')}")
        print(f"  Side: {order.get('side')}, Action: {order.get('action')}")
        print(f"  Count: {order.get('initial_count')}")
        print(f"  Status: {order.get('status')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating order: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order creation: {e}")
        return None

# Example usage: Create a limit order
# Note: headers variable should already be defined with authentication details
try:
    # Example: Buy 10 YES contracts at 50 cents
    new_order = create_order(
        headers=headers,
        ticker="EXAMPLE-TICKER",  # Replace with actual ticker
        side="yes",
        action="buy",
        count=10,
        yes_price=50  # 50 cents
    )
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_order = None

#### Batch Create Orders

In [ ]:
"""
Cell 19: Batch Create Orders
Creates multiple orders at once in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def batch_create_orders(headers, orders_list):
    """
    Creates multiple orders at once in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        orders_list: List of order dictionaries, each containing:
            - ticker (required)
            - side: 'yes' or 'no' (required)
            - action: 'buy' or 'sell' (required)
            - count: Number of contracts (required, >= 1)
            - type: 'limit' or 'market' (optional, default 'limit')
            - yes_price: Yes price (1-99, optional)
            - no_price: No price (1-99, optional)
            - client_order_id: Custom order ID (optional)
            - And other order parameters...
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/batched"

    # Prepare payload
    payload = {"orders": orders_list}

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        batch_results = data.get('orders', [])

        # Store successful orders in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        successful_count = 0
        failed_count = 0

        for result in batch_results:
            if result.get('order'):
                successful_count += 1
                order = result['order']

                cursor.execute('''
                    INSERT OR REPLACE INTO orders (
                        order_id, user_id, client_order_id, ticker, side, action,
                        type, status, yes_price, no_price, yes_price_dollars,
                        no_price_dollars, fill_count, remaining_count, initial_count,
                        taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                        taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                        taker_fees_dollars, maker_fees_dollars, expiration_time,
                        created_time, last_update_time, self_trade_prevention_type,
                        order_group_id, cancel_order_on_pause
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    order.get('order_id'),
                    order.get('user_id'),
                    order.get('client_order_id'),
                    order.get('ticker'),
                    order.get('side'),
                    order.get('action'),
                    order.get('type'),
                    order.get('status'),
                    order.get('yes_price'),
                    order.get('no_price'),
                    order.get('yes_price_dollars'),
                    order.get('no_price_dollars'),
                    order.get('fill_count'),
                    order.get('remaining_count'),
                    order.get('initial_count'),
                    order.get('taker_fees'),
                    order.get('maker_fees'),
                    order.get('taker_fill_cost'),
                    order.get('maker_fill_cost'),
                    order.get('taker_fill_cost_dollars'),
                    order.get('maker_fill_cost_dollars'),
                    order.get('queue_position'),
                    order.get('taker_fees_dollars'),
                    order.get('maker_fees_dollars'),
                    order.get('expiration_time'),
                    order.get('created_time'),
                    order.get('last_update_time'),
                    order.get('self_trade_prevention_type'),
                    order.get('order_group_id'),
                    order.get('cancel_order_on_pause')
                ))
            elif result.get('error'):
                failed_count += 1

        conn.commit()
        conn.close()

        print(f"Batch Order Creation Results:")
        print(f"  Total Orders: {len(batch_results)}")
        print(f"  Successful: {successful_count}")
        print(f"  Failed: {failed_count}")

        for result in batch_results[:5]:  # Show first 5
            if result.get('order'):
                order = result['order']
                print(f"  ✓ Order ID: {order.get('order_id')}, Ticker: {order.get('ticker')}")
            elif result.get('error'):
                error = result['error']
                print(f"  ✗ Error: {error.get('message')}")

        if len(batch_results) > 5:
            print(f"  ... and {len(batch_results) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating batch orders: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing batch order creation: {e}")
        return None

# Example usage: Create multiple orders at once
# Note: headers variable should already be defined with authentication details
try:
    # Example batch of orders
    batch_orders = [
        {
            "ticker": "EXAMPLE-TICKER-1",  # Replace with actual ticker
            "side": "yes",
            "action": "buy",
            "count": 10,
            "type": "limit",
            "yes_price": 50
        },
        {
            "ticker": "EXAMPLE-TICKER-2",  # Replace with actual ticker
            "side": "no",
            "action": "sell",
            "count": 5,
            "type": "limit",
            "no_price": 30
        }
    ]

    batch_result = batch_create_orders(headers, batch_orders)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    batch_result = None

#### Batch Cancel Orders

In [ ]:
"""
Cell 20: Batch Cancel Orders
Cancels multiple orders at once in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def batch_cancel_orders(headers, order_ids):
    """
    Cancels multiple orders at once in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_ids: List of order IDs to cancel
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/batched"

    # Prepare payload
    payload = {"ids": order_ids}

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.delete(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        batch_results = data.get('orders', [])

        # Update database for cancelled orders
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        successful_count = 0
        failed_count = 0

        for result in batch_results:
            order_id = result.get('order_id')

            if result.get('order'):
                successful_count += 1
                order = result['order']

                # Update order status in database
                cursor.execute('''
                    UPDATE orders
                    SET status = ?, last_update_time = ?
                    WHERE order_id = ?
                ''', ('canceled', datetime.utcnow().isoformat(), order_id))

            elif result.get('error'):
                failed_count += 1

        conn.commit()
        conn.close()

        print(f"Batch Cancel Results:")
        print(f"  Total Orders: {len(batch_results)}")
        print(f"  Successfully Cancelled: {successful_count}")
        print(f"  Failed: {failed_count}")

        for result in batch_results[:5]:  # Show first 5
            order_id = result.get('order_id')
            if result.get('order'):
                order = result['order']
                reduced_by = result.get('reduced_by', 0)
                print(f"  ✓ Order ID: {order_id}, Status: Cancelled")
                if reduced_by:
                    print(f"    Reduced by: {reduced_by}")
            elif result.get('error'):
                error = result['error']
                print(f"  ✗ Order ID: {order_id}, Error: {error.get('message')}")

        if len(batch_results) > 5:
            print(f"  ... and {len(batch_results) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error cancelling batch orders: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing batch order cancellation: {e}")
        return None

# Example usage: Cancel multiple orders at once
# Note: headers variable should already be defined with authentication details
try:
    # Example: Replace these with actual order IDs to cancel
    order_ids_to_cancel = [
        "order-id-1",
        "order-id-2",
        "order-id-3"
    ]

    batch_cancel_result = batch_cancel_orders(headers, order_ids_to_cancel)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    batch_cancel_result = None

#### Get Queue Positions for Orders

In [ ]:
"""
Cell 21: Get Queue Positions for Orders
Retrieves queue positions for all user orders from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_queue_positions_for_orders(headers):
    """
    Fetches queue positions for all user orders from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/queue_positions"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        queue_positions = data.get('queue_positions', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for position in queue_positions:
            cursor.execute('''
                INSERT INTO queue_positions (order_id, market_ticker, queue_position)
                VALUES (?, ?, ?)
            ''', (
                position.get('order_id'),
                position.get('market_ticker'),
                position.get('queue_position')
            ))

            # Also update the queue position in the orders table
            cursor.execute('''
                UPDATE orders
                SET queue_position = ?
                WHERE order_id = ?
            ''', (position.get('queue_position'), position.get('order_id')))

        conn.commit()
        conn.close()

        print(f"Queue Positions Retrieved: {len(queue_positions)} positions")
        for position in queue_positions[:10]:  # Show first 10
            print(f"  - Order ID: {position.get('order_id')}")
            print(f"    Market: {position.get('market_ticker')}, Position: #{position.get('queue_position')}")
        if len(queue_positions) > 10:
            print(f"  ... and {len(queue_positions) - 10} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching queue positions: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing queue positions: {e}")
        return None

# Execute the function
# Note: headers variable should already be defined with authentication details
try:
    queue_positions_data = get_queue_positions_for_orders(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    queue_positions_data = None

#### Get Order

In [ ]:
"""
Cell 22: Get Order
Retrieves details of a specific order from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_order(headers, order_id):
    """
    Fetches a specific order from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_id: The order ID to retrieve
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/{order_id}"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order = data.get('order', {})

        # Store/update in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO orders (
                order_id, user_id, client_order_id, ticker, side, action,
                type, status, yes_price, no_price, yes_price_dollars,
                no_price_dollars, fill_count, remaining_count, initial_count,
                taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                taker_fees_dollars, maker_fees_dollars, expiration_time,
                created_time, last_update_time, self_trade_prevention_type,
                order_group_id, cancel_order_on_pause
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            order.get('order_id'),
            order.get('user_id'),
            order.get('client_order_id'),
            order.get('ticker'),
            order.get('side'),
            order.get('action'),
            order.get('type'),
            order.get('status'),
            order.get('yes_price'),
            order.get('no_price'),
            order.get('yes_price_dollars'),
            order.get('no_price_dollars'),
            order.get('fill_count'),
            order.get('remaining_count'),
            order.get('initial_count'),
            order.get('taker_fees'),
            order.get('maker_fees'),
            order.get('taker_fill_cost'),
            order.get('maker_fill_cost'),
            order.get('taker_fill_cost_dollars'),
            order.get('maker_fill_cost_dollars'),
            order.get('queue_position'),
            order.get('taker_fees_dollars'),
            order.get('maker_fees_dollars'),
            order.get('expiration_time'),
            order.get('created_time'),
            order.get('last_update_time'),
            order.get('self_trade_prevention_type'),
            order.get('order_group_id'),
            order.get('cancel_order_on_pause')
        ))

        conn.commit()
        conn.close()

        print(f"Order Details Retrieved:")
        print(f"  Order ID: {order.get('order_id')}")
        print(f"  Ticker: {order.get('ticker')}")
        print(f"  Side: {order.get('side')}, Action: {order.get('action')}")
        print(f"  Type: {order.get('type')}, Status: {order.get('status')}")
        print(f"  Initial Count: {order.get('initial_count')}")
        print(f"  Filled: {order.get('fill_count')}, Remaining: {order.get('remaining_count')}")
        if order.get('queue_position'):
            print(f"  Queue Position: #{order.get('queue_position')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching order: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order: {e}")
        return None

# Example usage: Get a specific order
# Note: You need to provide a valid order_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-id' with an actual order ID
    order_id = "your-order-id"  # This should be replaced with an actual ID
    order_details = get_order(headers, order_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    order_details = None

#### Cancel Order

In [ ]:
"""
Cell 23: Cancel Order
Cancels a specific order in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def cancel_order(headers, order_id):
    """
    Cancels a specific order in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_id: The order ID to cancel
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/{order_id}"

    try:
        # Make the API request with authentication headers
        response = requests.delete(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order = data.get('order', {})
        reduced_by = data.get('reduced_by', 0)

        # Update database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Update order status to cancelled
        cursor.execute('''
            UPDATE orders
            SET status = ?, last_update_time = ?
            WHERE order_id = ?
        ''', ('canceled', datetime.utcnow().isoformat(), order_id))

        conn.commit()
        conn.close()

        print(f"Order Cancelled Successfully!")
        print(f"  Order ID: {order.get('order_id')}")
        print(f"  Ticker: {order.get('ticker')}")
        print(f"  Side: {order.get('side')}, Action: {order.get('action')}")
        print(f"  Final Status: {order.get('status')}")
        if reduced_by:
            print(f"  Reduced by: {reduced_by}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error cancelling order: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order cancellation: {e}")
        return None

# Example usage: Cancel a specific order
# Note: You need to provide a valid order_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-id' with an actual order ID to cancel
    order_id = "your-order-id"  # This should be replaced with an actual ID
    cancellation_result = cancel_order(headers, order_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    cancellation_result = None

#### Amend Order

In [ ]:
"""
Cell 24: Amend Order
Amends an existing order in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def amend_order(headers, order_id, ticker=None, side=None, action=None,
                client_order_id=None, updated_client_order_id=None,
                yes_price=None, no_price=None, yes_price_dollars=None,
                no_price_dollars=None, count=None):
    """
    Amends an existing order in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_id: The order ID to amend
        ticker: Market ticker (optional)
        side: 'yes' or 'no' (optional)
        action: 'buy' or 'sell' (optional)
        client_order_id: Current client order ID (optional)
        updated_client_order_id: New client order ID (optional)
        yes_price: New yes price (1-99, optional)
        no_price: New no price (1-99, optional)
        yes_price_dollars: New yes price in dollars (optional)
        no_price_dollars: New no price in dollars (optional)
        count: New order count (optional, >= 1)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/{order_id}/amend"

    # Build payload with only provided parameters
    payload = {}
    if ticker is not None:
        payload["ticker"] = ticker
    if side is not None:
        payload["side"] = side
    if action is not None:
        payload["action"] = action
    if client_order_id is not None:
        payload["client_order_id"] = client_order_id
    if updated_client_order_id is not None:
        payload["updated_client_order_id"] = updated_client_order_id
    if yes_price is not None:
        payload["yes_price"] = yes_price
    if no_price is not None:
        payload["no_price"] = no_price
    if yes_price_dollars is not None:
        payload["yes_price_dollars"] = yes_price_dollars
    if no_price_dollars is not None:
        payload["no_price_dollars"] = no_price_dollars
    if count is not None:
        payload["count"] = count

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        old_order = data.get('old_order', {})
        new_order = data.get('order', {})

        # Update database with new order information
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Store the amended order (new version)
        cursor.execute('''
            INSERT OR REPLACE INTO orders (
                order_id, user_id, client_order_id, ticker, side, action,
                type, status, yes_price, no_price, yes_price_dollars,
                no_price_dollars, fill_count, remaining_count, initial_count,
                taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                taker_fees_dollars, maker_fees_dollars, expiration_time,
                created_time, last_update_time, self_trade_prevention_type,
                order_group_id, cancel_order_on_pause
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            new_order.get('order_id'),
            new_order.get('user_id'),
            new_order.get('client_order_id'),
            new_order.get('ticker'),
            new_order.get('side'),
            new_order.get('action'),
            new_order.get('type'),
            new_order.get('status'),
            new_order.get('yes_price'),
            new_order.get('no_price'),
            new_order.get('yes_price_dollars'),
            new_order.get('no_price_dollars'),
            new_order.get('fill_count'),
            new_order.get('remaining_count'),
            new_order.get('initial_count'),
            new_order.get('taker_fees'),
            new_order.get('maker_fees'),
            new_order.get('taker_fill_cost'),
            new_order.get('maker_fill_cost'),
            new_order.get('taker_fill_cost_dollars'),
            new_order.get('maker_fill_cost_dollars'),
            new_order.get('queue_position'),
            new_order.get('taker_fees_dollars'),
            new_order.get('maker_fees_dollars'),
            new_order.get('expiration_time'),
            new_order.get('created_time'),
            new_order.get('last_update_time'),
            new_order.get('self_trade_prevention_type'),
            new_order.get('order_group_id'),
            new_order.get('cancel_order_on_pause')
        ))

        conn.commit()
        conn.close()

        print(f"Order Amended Successfully!")
        print(f"  Order ID: {new_order.get('order_id')}")
        print(f"  Old Order:")
        print(f"    Price: YES {old_order.get('yes_price')}, NO {old_order.get('no_price')}")
        print(f"    Count: {old_order.get('initial_count')}")
        print(f"  New Order:")
        print(f"    Price: YES {new_order.get('yes_price')}, NO {new_order.get('no_price')}")
        print(f"    Count: {new_order.get('initial_count')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error amending order: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order amendment: {e}")
        return None

# Example usage: Amend an order's price
# Note: You need to provide a valid order_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-id' with an actual order ID to amend
    order_id = "your-order-id"  # This should be replaced with an actual ID
    # Example: Change the yes price to 60 cents
    amended_order = amend_order(headers, order_id, yes_price=60)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    amended_order = None

#### Decrease Order

In [ ]:
"""
Cell 25: Decrease Order
Decreases the size of an existing order in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def decrease_order(headers, order_id, reduce_by=None, reduce_to=None):
    """
    Decreases the size of an existing order in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_id: The order ID to decrease
        reduce_by: Number of contracts to reduce by (optional)
        reduce_to: Number of contracts to reduce to (optional)

    Note: Provide either reduce_by OR reduce_to, not both
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/{order_id}/decrease"

    # Build payload
    payload = {}
    if reduce_by is not None:
        payload["reduce_by"] = reduce_by
    if reduce_to is not None:
        payload["reduce_to"] = reduce_to

    if not payload:
        print("Error: Must provide either reduce_by or reduce_to parameter")
        return None

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        order = data.get('order', {})

        # Update database with decreased order
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO orders (
                order_id, user_id, client_order_id, ticker, side, action,
                type, status, yes_price, no_price, yes_price_dollars,
                no_price_dollars, fill_count, remaining_count, initial_count,
                taker_fees, maker_fees, taker_fill_cost, maker_fill_cost,
                taker_fill_cost_dollars, maker_fill_cost_dollars, queue_position,
                taker_fees_dollars, maker_fees_dollars, expiration_time,
                created_time, last_update_time, self_trade_prevention_type,
                order_group_id, cancel_order_on_pause
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            order.get('order_id'),
            order.get('user_id'),
            order.get('client_order_id'),
            order.get('ticker'),
            order.get('side'),
            order.get('action'),
            order.get('type'),
            order.get('status'),
            order.get('yes_price'),
            order.get('no_price'),
            order.get('yes_price_dollars'),
            order.get('no_price_dollars'),
            order.get('fill_count'),
            order.get('remaining_count'),
            order.get('initial_count'),
            order.get('taker_fees'),
            order.get('maker_fees'),
            order.get('taker_fill_cost'),
            order.get('maker_fill_cost'),
            order.get('taker_fill_cost_dollars'),
            order.get('maker_fill_cost_dollars'),
            order.get('queue_position'),
            order.get('taker_fees_dollars'),
            order.get('maker_fees_dollars'),
            order.get('expiration_time'),
            order.get('created_time'),
            order.get('last_update_time'),
            order.get('self_trade_prevention_type'),
            order.get('order_group_id'),
            order.get('cancel_order_on_pause')
        ))

        conn.commit()
        conn.close()

        print(f"Order Decreased Successfully!")
        print(f"  Order ID: {order.get('order_id')}")
        print(f"  Ticker: {order.get('ticker')}")
        print(f"  Updated Count:")
        print(f"    Initial: {order.get('initial_count')}")
        print(f"    Remaining: {order.get('remaining_count')}")
        print(f"    Filled: {order.get('fill_count')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error decreasing order: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order decrease: {e}")
        return None

# Example usage: Decrease an order size
# Note: You need to provide a valid order_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-id' with an actual order ID to decrease
    order_id = "your-order-id"  # This should be replaced with an actual ID

    # Example 1: Reduce by 5 contracts
    # decreased_order = decrease_order(headers, order_id, reduce_by=5)

    # Example 2: Reduce to 10 contracts total
    decreased_order = decrease_order(headers, order_id, reduce_to=10)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    decreased_order = None

#### Get Order Queue Position

In [ ]:
"""
Cell 26: Get Order Queue Position
Retrieves the queue position for a specific order from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_order_queue_position(headers, order_id):
    """
    Fetches the queue position for a specific order from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        order_id: The order ID to get queue position for
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/portfolio/orders/{order_id}/queue_position"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        queue_position = data.get('queue_position')

        # Update database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Update the queue position in the orders table
        cursor.execute('''
            UPDATE orders
            SET queue_position = ?, last_update_time = ?
            WHERE order_id = ?
        ''', (queue_position, datetime.utcnow().isoformat(), order_id))

        # Also insert/update in queue_positions table
        cursor.execute('''
            INSERT OR REPLACE INTO queue_positions (order_id, queue_position)
            VALUES (?, ?)
        ''', (order_id, queue_position))

        conn.commit()
        conn.close()

        print(f"Order Queue Position Retrieved:")
        print(f"  Order ID: {order_id}")
        print(f"  Queue Position: #{queue_position}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching order queue position: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing order queue position: {e}")
        return None

# Example usage: Get queue position for a specific order
# Note: You need to provide a valid order_id
# Note: headers variable should already be defined with authentication details
try:
    # Replace 'your-order-id' with an actual order ID
    order_id = "your-order-id"  # This should be replaced with an actual ID
    queue_position = get_order_queue_position(headers, order_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    queue_position = None

## **API Keys**

#### Get API Keys

In [ ]:
"""
Cell 28: Get API Keys
Retrieves list of API keys from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_api_keys(headers):
    """
    Fetches list of API keys from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/api_keys"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        api_keys = data.get('api_keys', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for key in api_keys:
            cursor.execute('''
                INSERT OR REPLACE INTO api_keys (api_key_id, name)
                VALUES (?, ?)
            ''', (
                key.get('api_key_id'),
                key.get('name')
            ))

        conn.commit()
        conn.close()

        print(f"API Keys Retrieved: {len(api_keys)} keys")
        for key in api_keys:
            print(f"  - ID: {key.get('api_key_id')}")
            print(f"    Name: {key.get('name')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching API keys: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing API keys: {e}")
        return None

# Execute the function
# Note: headers variable should already be defined with authentication details
try:
    api_keys_data = get_api_keys(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    api_keys_data = None

#### Create API Key

In [ ]:
"""
Cell 29: Create API Key
Creates a new API key in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def create_api_key(headers, name, public_key):
    """
    Creates a new API key in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        name: Name for the API key
        public_key: Public key for the API key
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/api_keys"

    # Prepare payload
    payload = {
        "name": name,
        "public_key": public_key
    }

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        api_key_id = data.get('api_key_id')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO api_keys (api_key_id, name)
            VALUES (?, ?)
        ''', (api_key_id, name))

        conn.commit()
        conn.close()

        print(f"API Key Created Successfully!")
        print(f"  API Key ID: {api_key_id}")
        print(f"  Name: {name}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating API key: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing API key creation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual values
    name = "My New API Key"
    public_key = "your-public-key-here"

    new_api_key = create_api_key(headers, name, public_key)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_api_key = None

#### Generate API Key

In [ ]:
"""
Cell 30: Generate API Key
Generates a new API key with private key in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def generate_api_key(headers, name):
    """
    Generates a new API key with private key in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        name: Name for the API key
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/api_keys/generate"

    # Prepare payload
    payload = {"name": name}

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        api_key_id = data.get('api_key_id')
        private_key = data.get('private_key')

        # Store in database (Note: Be careful with private key storage in production!)
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO api_keys (api_key_id, name, private_key)
            VALUES (?, ?, ?)
        ''', (api_key_id, name, private_key))

        conn.commit()
        conn.close()

        print(f"API Key Generated Successfully!")
        print(f"  API Key ID: {api_key_id}")
        print(f"  Name: {name}")
        print(f"  Private Key: {'*' * 20} (stored securely)")
        print(f"\n⚠️ WARNING: Store the private key securely! It cannot be retrieved again.")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error generating API key: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing API key generation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with your desired API key name
    name = "Generated API Key"

    generated_key = generate_api_key(headers, name)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    generated_key = None

#### Delete API Key

In [ ]:
"""
Cell 31: Delete API Key
Deletes an API key from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def delete_api_key(headers, api_key):
    """
    Deletes an API key from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        api_key: The API key ID to delete
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/api_keys/{api_key}"

    try:
        # Make the API request with authentication headers
        response = requests.delete(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Remove from database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('DELETE FROM api_keys WHERE api_key_id = ?', (api_key,))

        conn.commit()
        conn.close()

        print(f"API Key Deleted Successfully!")
        print(f"  API Key ID: {api_key}")

        return data if data else {"success": True, "deleted_id": api_key}

    except requests.exceptions.RequestException as e:
        print(f"Error deleting API key: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing API key deletion: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual API key ID to delete
    api_key_to_delete = "your-api-key-id"

    deletion_result = delete_api_key(headers, api_key_to_delete)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    deletion_result = None

## **Search**

#### Get Tags for Series Categories

In [ ]:
"""
Cell 32: Get Tags for Series Categories
Retrieves tags organized by categories from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_tags_by_categories():
    """
    Fetches tags organized by categories from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/search/tags_by_categories"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        tags_by_categories = data.get('tags_by_categories', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for category, tags in tags_by_categories.items():
            cursor.execute('''
                INSERT INTO tags_by_categories (category, tags)
                VALUES (?, ?)
            ''', (category, json.dumps(tags)))

        conn.commit()
        conn.close()

        print(f"Tags by Categories Retrieved: {len(tags_by_categories)} categories")
        for category, tags in list(tags_by_categories.items())[:5]:
            print(f"  - {category}: {len(tags) if isinstance(tags, list) else 'N/A'} tags")
            if isinstance(tags, list) and tags:
                print(f"    Sample tags: {', '.join(tags[:3])}")
        if len(tags_by_categories) > 5:
            print(f"  ... and {len(tags_by_categories) - 5} more categories")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching tags by categories: {e}")
        return None
    except Exception as e:
        print(f"Error processing tags by categories: {e}")
        return None

# Execute the function
tags_data = get_tags_by_categories()

#### Get Filters for Sports

In [ ]:
"""
Cell 33: Get Filters for Sports
Retrieves filters organized by sport from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_filters_by_sport():
    """
    Fetches filters organized by sport from Kalshi API
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/search/filters_by_sport"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        filters_by_sports = data.get('filters_by_sports', {})
        sport_ordering = data.get('sport_ordering', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for idx, sport in enumerate(sport_ordering):
            filters = filters_by_sports.get(sport, {})
            cursor.execute('''
                INSERT INTO sports_filters (sport, filters, sport_order)
                VALUES (?, ?, ?)
            ''', (sport, json.dumps(filters), idx))

        conn.commit()
        conn.close()

        print(f"Sports Filters Retrieved: {len(filters_by_sports)} sports")
        print(f"Sport Ordering: {', '.join(sport_ordering[:5])}")
        if len(sport_ordering) > 5:
            print(f"  ... and {len(sport_ordering) - 5} more sports")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching sports filters: {e}")
        return None
    except Exception as e:
        print(f"Error processing sports filters: {e}")
        return None

# Execute the function
sports_filters_data = get_filters_by_sport()

## **Market**

#### Get Market Candlesticks

In [ ]:
"""
Cell 34: Get Market Candlesticks
Retrieves candlestick data for a specific market
"""

import requests
import sqlite3
from datetime import datetime

def get_market_candlesticks(series_ticker, ticker, start_ts, end_ts, period_interval):
    """
    Fetches candlestick data for a specific market from Kalshi API

    Args:
        series_ticker: Series ticker that contains the target market
        ticker: Market ticker - unique identifier for the specific market
        start_ts: Start timestamp (Unix timestamp)
        end_ts: End timestamp (Unix timestamp)
        period_interval: Time period length (1, 60, or 1440 minutes)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/series/{series_ticker}/markets/{ticker}/candlesticks"

    # Build query parameters
    params = {
        "start_ts": start_ts,
        "end_ts": end_ts,
        "period_interval": period_interval
    }

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        candlesticks = data.get('candlesticks', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for candle in candlesticks:
            yes_bid = candle.get('yes_bid', {})
            yes_ask = candle.get('yes_ask', {})
            price = candle.get('price', {})

            cursor.execute('''
                INSERT INTO candlesticks (
                    ticker, end_period_ts, period_interval,
                    yes_bid_open, yes_bid_low, yes_bid_high, yes_bid_close,
                    yes_ask_open, yes_ask_low, yes_ask_high, yes_ask_close,
                    price_open, price_low, price_high, price_close,
                    price_mean, price_previous, volume, open_interest
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                ticker,
                candle.get('end_period_ts'),
                period_interval,
                yes_bid.get('open'), yes_bid.get('low'), yes_bid.get('high'), yes_bid.get('close'),
                yes_ask.get('open'), yes_ask.get('low'), yes_ask.get('high'), yes_ask.get('close'),
                price.get('open'), price.get('low'), price.get('high'), price.get('close'),
                price.get('mean'), price.get('previous'),
                candle.get('volume'),
                candle.get('open_interest')
            ))

        conn.commit()
        conn.close()

        print(f"Market Candlesticks Retrieved: {len(candlesticks)} candlesticks")
        print(f"  Ticker: {ticker}")
        print(f"  Period: {period_interval} minutes")
        if candlesticks:
            print(f"  First: {datetime.fromtimestamp(candlesticks[0].get('end_period_ts', 0))}")
            print(f"  Last: {datetime.fromtimestamp(candlesticks[-1].get('end_period_ts', 0))}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching market candlesticks: {e}")
        return None
    except Exception as e:
        print(f"Error processing market candlesticks: {e}")
        return None

# Example usage
# Replace with actual values
series_ticker = "EXAMPLE-SERIES"
market_ticker = "EXAMPLE-MARKET"
start_timestamp = 1698768000  # Example timestamp
end_timestamp = 1698854400    # Example timestamp
period = 60  # 1 hour candles

candlesticks_data = get_market_candlesticks(series_ticker, market_ticker, start_timestamp, end_timestamp, period)

#### Get Trades

In [ ]:
"""
Cell 35: Get Trades
Retrieves trades data from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_trades(limit=100, cursor=None, ticker=None, min_ts=None, max_ts=None):
    """
    Fetches trades data from Kalshi API

    Args:
        limit: Number of results per page (1-1000, default 100)
        cursor: Pagination cursor from previous response
        ticker: Filter by market ticker
        min_ts: Filter items after this Unix timestamp
        max_ts: Filter items before this Unix timestamp
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/markets/trades"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if ticker:
        params["ticker"] = ticker
    if min_ts:
        params["min_ts"] = min_ts
    if max_ts:
        params["max_ts"] = max_ts

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        trades = data.get('trades', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for trade in trades:
            cursor_db.execute('''
                INSERT OR REPLACE INTO trades (
                    trade_id, ticker, price, count, yes_price, no_price,
                    taker_side, created_time
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                trade.get('trade_id'),
                trade.get('ticker'),
                trade.get('price'),
                trade.get('count'),
                trade.get('yes_price'),
                trade.get('no_price'),
                trade.get('taker_side'),
                trade.get('created_time')
            ))

        conn.commit()
        conn.close()

        print(f"Trades Retrieved: {len(trades)} trades")
        for trade in trades[:5]:  # Show first 5
            print(f"  - Trade ID: {trade.get('trade_id')}")
            print(f"    Ticker: {trade.get('ticker')}, Side: {trade.get('taker_side')}")
            print(f"    Price: {trade.get('price')}, Count: {trade.get('count')}")
        if len(trades) > 5:
            print(f"  ... and {len(trades) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching trades: {e}")
        return None
    except Exception as e:
        print(f"Error processing trades: {e}")
        return None

# Execute the function with default parameters
trades_data = get_trades()

#### Get Market Orderbook

In [ ]:
"""
Cell 36: Get Market Orderbook
Retrieves the orderbook for a specific market (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_market_orderbook(headers, ticker):
    """
    Fetches the orderbook for a specific market from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        ticker: Market ticker
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{ticker}/orderbook"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        orderbook = data.get('orderbook', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Clear existing orderbook data for this ticker
        cursor.execute('DELETE FROM orderbook WHERE ticker = ?', (ticker,))

        # Store yes side orders
        yes_orders = orderbook.get('yes', [])
        for order in yes_orders:
            if len(order) >= 2:  # Ensure we have [price, quantity]
                cursor.execute('''
                    INSERT INTO orderbook (ticker, side, price_level, quantity)
                    VALUES (?, ?, ?, ?)
                ''', (ticker, 'yes', order[0], order[1] if len(order) > 1 else 0))

        # Store no side orders
        no_orders = orderbook.get('no', [])
        for order in no_orders:
            if len(order) >= 2:  # Ensure we have [price, quantity]
                cursor.execute('''
                    INSERT INTO orderbook (ticker, side, price_level, quantity)
                    VALUES (?, ?, ?, ?)
                ''', (ticker, 'no', order[0], order[1] if len(order) > 1 else 0))

        conn.commit()
        conn.close()

        print(f"Market Orderbook Retrieved for {ticker}:")
        print(f"  YES side: {len(yes_orders)} levels")
        if yes_orders and yes_orders[0]:
            print(f"    Best YES: Price {yes_orders[0][0]} with quantity {yes_orders[0][1] if len(yes_orders[0]) > 1 else 'N/A'}")
        print(f"  NO side: {len(no_orders)} levels")
        if no_orders and no_orders[0]:
            print(f"    Best NO: Price {no_orders[0][0]} with quantity {no_orders[0][1] if len(no_orders[0]) > 1 else 'N/A'}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching market orderbook: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing market orderbook: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual market ticker
    market_ticker = "EXAMPLE-MARKET"

    orderbook_data = get_market_orderbook(headers, market_ticker)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    orderbook_data = None

#### Get Series

In [ ]:
"""
Cell 37: Get Series
Retrieves information about a specific series from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_series(series_ticker):
    """
    Fetches information about a specific series from Kalshi API

    Args:
        series_ticker: The ticker of the series to retrieve
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/series/{series_ticker}"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        series = data.get('series', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO series (
                ticker, frequency, title, category, tags, settlement_sources,
                contract_url, contract_terms_url, product_metadata,
                fee_type, fee_multiplier, additional_prohibitions
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            series.get('ticker'),
            series.get('frequency'),
            series.get('title'),
            series.get('category'),
            json.dumps(series.get('tags', [])),
            json.dumps(series.get('settlement_sources', [])),
            series.get('contract_url'),
            series.get('contract_terms_url'),
            json.dumps(series.get('product_metadata', {})),
            series.get('fee_type'),
            series.get('fee_multiplier'),
            json.dumps(series.get('additional_prohibitions', []))
        ))

        conn.commit()
        conn.close()

        print(f"Series Retrieved: {series.get('ticker')}")
        print(f"  Title: {series.get('title')}")
        print(f"  Category: {series.get('category')}")
        print(f"  Frequency: {series.get('frequency')}")
        print(f"  Fee Type: {series.get('fee_type')}")
        tags = series.get('tags', [])
        if tags:
            print(f"  Tags: {', '.join(tags[:5])}")
            if len(tags) > 5:
                print(f"    ... and {len(tags) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching series: {e}")
        return None
    except Exception as e:
        print(f"Error processing series: {e}")
        return None

# Example usage
# Replace with actual series ticker
series_ticker = "EXAMPLE-SERIES"

series_data = get_series(series_ticker)

#### Get Series List

In [ ]:
"""
Cell 38: Get Series List
Retrieves list of all series from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_series_list(category=None, tags=None, include_product_metadata=False):
    """
    Fetches list of all series from Kalshi API

    Args:
        category: Filter by category (optional)
        tags: Filter by tags (optional)
        include_product_metadata: Include product metadata (default False)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/series"

    # Build query parameters
    params = {}
    if category:
        params["category"] = category
    if tags:
        params["tags"] = tags
    if include_product_metadata:
        params["include_product_metadata"] = include_product_metadata

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        series_list = data.get('series', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for series in series_list:
            cursor.execute('''
                INSERT OR REPLACE INTO series (
                    ticker, frequency, title, category, tags, settlement_sources,
                    contract_url, contract_terms_url, product_metadata,
                    fee_type, fee_multiplier, additional_prohibitions
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                series.get('ticker'),
                series.get('frequency'),
                series.get('title'),
                series.get('category'),
                json.dumps(series.get('tags', [])),
                json.dumps(series.get('settlement_sources', [])),
                series.get('contract_url'),
                series.get('contract_terms_url'),
                json.dumps(series.get('product_metadata', {})),
                series.get('fee_type'),
                series.get('fee_multiplier'),
                json.dumps(series.get('additional_prohibitions', []))
            ))

        conn.commit()
        conn.close()

        print(f"Series List Retrieved: {len(series_list)} series")
        for series in series_list[:5]:  # Show first 5
            print(f"  - {series.get('ticker')}: {series.get('title')}")
            print(f"    Category: {series.get('category')}, Frequency: {series.get('frequency')}")
        if len(series_list) > 5:
            print(f"  ... and {len(series_list) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching series list: {e}")
        return None
    except Exception as e:
        print(f"Error processing series list: {e}")
        return None

# Execute the function with default parameters
series_list_data = get_series_list()

#### Get Markets

In [ ]:
"""
Cell 39: Get Markets
Retrieves list of markets from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_markets(limit=100, cursor=None, event_ticker=None, series_ticker=None,
                max_close_ts=None, min_close_ts=None, status=None, tickers=None,
                mve_filter=None):
    """
    Fetches list of markets from Kalshi API

    Args:
        limit: Number of results per page (1-1000, default 100)
        cursor: Pagination cursor from previous response
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        series_ticker: Filter by series ticker
        max_close_ts: Filter items that close before this Unix timestamp
        min_close_ts: Filter items that close after this Unix timestamp
        status: Filter by status (comma-separated: unopened, open, closed, settled)
        tickers: Filter by specific market tickers (comma-separated)
        mve_filter: Filter multivariate events ('only' or 'exclude')
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/markets"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if event_ticker:
        params["event_ticker"] = event_ticker
    if series_ticker:
        params["series_ticker"] = series_ticker
    if max_close_ts:
        params["max_close_ts"] = max_close_ts
    if min_close_ts:
        params["min_close_ts"] = min_close_ts
    if status:
        params["status"] = status
    if tickers:
        params["tickers"] = tickers
    if mve_filter:
        params["mve_filter"] = mve_filter

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        markets = data.get('markets', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for market in markets:
            cursor_db.execute('''
                INSERT OR REPLACE INTO markets (
                    ticker, event_ticker, market_type, title, subtitle,
                    yes_sub_title, no_sub_title, open_time, close_time,
                    expected_expiration_time, expiration_time, latest_expiration_time,
                    settlement_timer_seconds, status, response_price_units,
                    yes_bid, yes_ask, no_bid, no_ask, last_price,
                    volume, volume_24h, result, can_close_early, open_interest,
                    notional_value, liquidity, settlement_value, category,
                    risk_limit_cents, fee_waiver_expiration_time, tick_size,
                    strike_type, floor_strike, cap_strike, rules_primary,
                    rules_secondary, mve_collection_ticker
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                market.get('ticker'),
                market.get('event_ticker'),
                market.get('market_type'),
                market.get('title'),
                market.get('subtitle'),
                market.get('yes_sub_title'),
                market.get('no_sub_title'),
                market.get('open_time'),
                market.get('close_time'),
                market.get('expected_expiration_time'),
                market.get('expiration_time'),
                market.get('latest_expiration_time'),
                market.get('settlement_timer_seconds'),
                market.get('status'),
                market.get('response_price_units'),
                market.get('yes_bid'),
                market.get('yes_ask'),
                market.get('no_bid'),
                market.get('no_ask'),
                market.get('last_price'),
                market.get('volume'),
                market.get('volume_24h'),
                market.get('result'),
                market.get('can_close_early'),
                market.get('open_interest'),
                market.get('notional_value'),
                market.get('liquidity'),
                market.get('settlement_value'),
                market.get('category'),
                market.get('risk_limit_cents'),
                market.get('fee_waiver_expiration_time'),
                market.get('tick_size'),
                market.get('strike_type'),
                market.get('floor_strike'),
                market.get('cap_strike'),
                market.get('rules_primary'),
                market.get('rules_secondary'),
                market.get('mve_collection_ticker')
            ))

        conn.commit()
        conn.close()

        print(f"Markets Retrieved: {len(markets)} markets")
        for market in markets[:5]:  # Show first 5
            print(f"  - {market.get('ticker')}: {market.get('title')}")
            print(f"    Status: {market.get('status')}, Volume: {market.get('volume')}")
        if len(markets) > 5:
            print(f"  ... and {len(markets) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching markets: {e}")
        return None
    except Exception as e:
        print(f"Error processing markets: {e}")
        return None

# Execute the function with default parameters
markets_data = get_markets()

#### Get Market

In [ ]:
"""
Cell 40: Get Market
Retrieves information about a specific market from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_market(ticker):
    """
    Fetches information about a specific market from Kalshi API

    Args:
        ticker: Market ticker
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{ticker}"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        market = data.get('market', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO markets (
                ticker, event_ticker, market_type, title, subtitle,
                yes_sub_title, no_sub_title, open_time, close_time,
                expected_expiration_time, expiration_time, latest_expiration_time,
                settlement_timer_seconds, status, response_price_units,
                yes_bid, yes_ask, no_bid, no_ask, last_price,
                volume, volume_24h, result, can_close_early, open_interest,
                notional_value, liquidity, settlement_value, category,
                risk_limit_cents, fee_waiver_expiration_time, tick_size,
                strike_type, floor_strike, cap_strike, rules_primary,
                rules_secondary, mve_collection_ticker
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            market.get('ticker'),
            market.get('event_ticker'),
            market.get('market_type'),
            market.get('title'),
            market.get('subtitle'),
            market.get('yes_sub_title'),
            market.get('no_sub_title'),
            market.get('open_time'),
            market.get('close_time'),
            market.get('expected_expiration_time'),
            market.get('expiration_time'),
            market.get('latest_expiration_time'),
            market.get('settlement_timer_seconds'),
            market.get('status'),
            market.get('response_price_units'),
            market.get('yes_bid'),
            market.get('yes_ask'),
            market.get('no_bid'),
            market.get('no_ask'),
            market.get('last_price'),
            market.get('volume'),
            market.get('volume_24h'),
            market.get('result'),
            market.get('can_close_early'),
            market.get('open_interest'),
            market.get('notional_value'),
            market.get('liquidity'),
            market.get('settlement_value'),
            market.get('category'),
            market.get('risk_limit_cents'),
            market.get('fee_waiver_expiration_time'),
            market.get('tick_size'),
            market.get('strike_type'),
            market.get('floor_strike'),
            market.get('cap_strike'),
            market.get('rules_primary'),
            market.get('rules_secondary'),
            market.get('mve_collection_ticker')
        ))

        conn.commit()
        conn.close()

        print(f"Market Retrieved: {market.get('ticker')}")
        print(f"  Title: {market.get('title')}")
        print(f"  Status: {market.get('status')}")
        print(f"  Yes Bid/Ask: {market.get('yes_bid')}/{market.get('yes_ask')}")
        print(f"  No Bid/Ask: {market.get('no_bid')}/{market.get('no_ask')}")
        print(f"  Volume: {market.get('volume')}")
        print(f"  Open Interest: {market.get('open_interest')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching market: {e}")
        return None
    except Exception as e:
        print(f"Error processing market: {e}")
        return None

# Example usage
# Replace with actual market ticker
market_ticker = "EXAMPLE-MARKET"

market_data = get_market(market_ticker)

## **Events**

#### Get Event Candlesticks

In [ ]:
"""
Cell 41: Get Event Candlesticks
Retrieves candlestick data for a specific event
"""

import requests
import sqlite3
from datetime import datetime

def get_event_candlesticks(series_ticker, event_ticker, start_ts, end_ts, period_interval):
    """
    Fetches candlestick data for a specific event from Kalshi API

    Args:
        series_ticker: The series ticker
        event_ticker: The event ticker
        start_ts: Start timestamp (Unix timestamp)
        end_ts: End timestamp (Unix timestamp)
        period_interval: Time period length (1, 60, or 1440 minutes)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/series/{series_ticker}/events/{event_ticker}/candlesticks"

    # Build query parameters
    params = {
        "start_ts": start_ts,
        "end_ts": end_ts,
        "period_interval": period_interval
    }

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        market_tickers = data.get('market_tickers', [])
        market_candlesticks = data.get('market_candlesticks', [])
        adjusted_end_ts = data.get('adjusted_end_ts')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Process each market's candlesticks
        for i, ticker in enumerate(market_tickers):
            if i < len(market_candlesticks):
                candlesticks = market_candlesticks[i]
                for candle in candlesticks:
                    yes_bid = candle.get('yes_bid', {})
                    yes_ask = candle.get('yes_ask', {})
                    price = candle.get('price', {})

                    cursor.execute('''
                        INSERT INTO candlesticks (
                            ticker, end_period_ts, period_interval,
                            yes_bid_open, yes_bid_low, yes_bid_high, yes_bid_close,
                            yes_ask_open, yes_ask_low, yes_ask_high, yes_ask_close,
                            price_open, price_low, price_high, price_close,
                            price_mean, price_previous, volume, open_interest
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        ticker,
                        candle.get('end_period_ts'),
                        period_interval,
                        yes_bid.get('open'), yes_bid.get('low'), yes_bid.get('high'), yes_bid.get('close'),
                        yes_ask.get('open'), yes_ask.get('low'), yes_ask.get('high'), yes_ask.get('close'),
                        price.get('open'), price.get('low'), price.get('high'), price.get('close'),
                        price.get('mean'), price.get('previous'),
                        candle.get('volume'),
                        candle.get('open_interest')
                    ))

        conn.commit()
        conn.close()

        print(f"Event Candlesticks Retrieved for {event_ticker}:")
        print(f"  Markets: {len(market_tickers)}")
        print(f"  Period: {period_interval} minutes")
        if market_tickers:
            print(f"  Market Tickers: {', '.join(market_tickers[:3])}")
            if len(market_tickers) > 3:
                print(f"    ... and {len(market_tickers) - 3} more")
        if adjusted_end_ts:
            print(f"  Adjusted End Time: {datetime.fromtimestamp(adjusted_end_ts)}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching event candlesticks: {e}")
        return None
    except Exception as e:
        print(f"Error processing event candlesticks: {e}")
        return None

# Example usage
series_ticker = "EXAMPLE-SERIES"
event_ticker = "EXAMPLE-EVENT"
start_timestamp = 1698768000  # Example timestamp
end_timestamp = 1698854400    # Example timestamp
period = 60  # 1 hour candles

event_candlesticks_data = get_event_candlesticks(series_ticker, event_ticker, start_timestamp, end_timestamp, period)

#### Get Events

In [ ]:
"""
Cell 42: Get Events
Retrieves list of events from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_events(limit=200, cursor=None, with_nested_markets=False, with_milestones=False,
                status=None, series_ticker=None, min_close_ts=None):
    """
    Fetches list of events from Kalshi API

    Args:
        limit: Number of results per page (1-200, default 200)
        cursor: Pagination cursor from previous response
        with_nested_markets: Include nested markets in response (default False)
        with_milestones: Include related milestones (default False)
        status: Filter by status ('open', 'closed', 'settled')
        series_ticker: Filter by series ticker
        min_close_ts: Filter events with at least one market closing after this timestamp
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/events"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if with_nested_markets:
        params["with_nested_markets"] = with_nested_markets
    if with_milestones:
        params["with_milestones"] = with_milestones
    if status:
        params["status"] = status
    if series_ticker:
        params["series_ticker"] = series_ticker
    if min_close_ts:
        params["min_close_ts"] = min_close_ts

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        events = data.get('events', [])
        milestones = data.get('milestones', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        # Store events
        for event in events:
            cursor_db.execute('''
                INSERT OR REPLACE INTO events (
                    event_ticker, series_ticker, sub_title, title,
                    collateral_return_type, mutually_exclusive, category,
                    strike_date, strike_period, available_on_brokers,
                    product_metadata
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                event.get('event_ticker'),
                event.get('series_ticker'),
                event.get('sub_title'),
                event.get('title'),
                event.get('collateral_return_type'),
                event.get('mutually_exclusive'),
                event.get('category'),
                event.get('strike_date'),
                event.get('strike_period'),
                event.get('available_on_brokers'),
                json.dumps(event.get('product_metadata', {}))
            ))

            # If markets are nested, store them too
            if with_nested_markets and 'markets' in event:
                for market in event.get('markets', []):
                    cursor_db.execute('''
                        INSERT OR REPLACE INTO markets (
                            ticker, event_ticker, market_type, title, subtitle,
                            status, volume, last_price
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        market.get('ticker'),
                        market.get('event_ticker'),
                        market.get('market_type'),
                        market.get('title'),
                        market.get('subtitle'),
                        market.get('status'),
                        market.get('volume'),
                        market.get('last_price')
                    ))

        # Store milestones if included
        for milestone in milestones:
            cursor_db.execute('''
                INSERT OR REPLACE INTO milestones (
                    id, category, type, start_date, end_date,
                    related_event_tickers, title, notification_message,
                    source_id, details, primary_event_tickers, last_updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                milestone.get('id'),
                milestone.get('category'),
                milestone.get('type'),
                milestone.get('start_date'),
                milestone.get('end_date'),
                json.dumps(milestone.get('related_event_tickers', [])),
                milestone.get('title'),
                milestone.get('notification_message'),
                milestone.get('source_id'),
                json.dumps(milestone.get('details', {})),
                json.dumps(milestone.get('primary_event_tickers', [])),
                milestone.get('last_updated_ts')
            ))

        conn.commit()
        conn.close()

        print(f"Events Retrieved: {len(events)} events")
        if milestones:
            print(f"Milestones Retrieved: {len(milestones)} milestones")
        for event in events[:5]:  # Show first 5
            print(f"  - {event.get('event_ticker')}: {event.get('title')}")
            print(f"    Category: {event.get('category')}")
        if len(events) > 5:
            print(f"  ... and {len(events) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching events: {e}")
        return None
    except Exception as e:
        print(f"Error processing events: {e}")
        return None

# Execute the function with default parameters
events_data = get_events()

#### Get Multivariate Events

In [ ]:
"""
Cell 43: Get Multivariate Events
Retrieves list of multivariate events from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_multivariate_events(limit=100, cursor=None, series_ticker=None,
                           collection_ticker=None, with_nested_markets=False):
    """
    Fetches list of multivariate events from Kalshi API

    Args:
        limit: Number of results per page (1-200, default 100)
        cursor: Pagination cursor from previous response
        series_ticker: Filter by series ticker
        collection_ticker: Filter by collection ticker (cannot use with series_ticker)
        with_nested_markets: Include nested markets in response (default False)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/events/multivariate"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if series_ticker:
        params["series_ticker"] = series_ticker
    if collection_ticker:
        params["collection_ticker"] = collection_ticker
    if with_nested_markets:
        params["with_nested_markets"] = with_nested_markets

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        events = data.get('events', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for event in events:
            cursor_db.execute('''
                INSERT OR REPLACE INTO events (
                    event_ticker, series_ticker, sub_title, title,
                    collateral_return_type, mutually_exclusive, category,
                    strike_date, strike_period, available_on_brokers,
                    product_metadata
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                event.get('event_ticker'),
                event.get('series_ticker'),
                event.get('sub_title'),
                event.get('title'),
                event.get('collateral_return_type'),
                event.get('mutually_exclusive'),
                event.get('category'),
                event.get('strike_date'),
                event.get('strike_period'),
                event.get('available_on_brokers'),
                json.dumps(event.get('product_metadata', {}))
            ))

            # If markets are nested, store them too
            if with_nested_markets and 'markets' in event:
                for market in event.get('markets', []):
                    cursor_db.execute('''
                        INSERT OR REPLACE INTO markets (
                            ticker, event_ticker, market_type, title, subtitle,
                            status, volume, last_price, mve_collection_ticker
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        market.get('ticker'),
                        market.get('event_ticker'),
                        market.get('market_type'),
                        market.get('title'),
                        market.get('subtitle'),
                        market.get('status'),
                        market.get('volume'),
                        market.get('last_price'),
                        market.get('mve_collection_ticker')
                    ))

        conn.commit()
        conn.close()

        print(f"Multivariate Events Retrieved: {len(events)} events")
        for event in events[:5]:  # Show first 5
            print(f"  - {event.get('event_ticker')}: {event.get('title')}")
            print(f"    Category: {event.get('category')}")
            print(f"    Mutually Exclusive: {event.get('mutually_exclusive')}")
        if len(events) > 5:
            print(f"  ... and {len(events) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching multivariate events: {e}")
        return None
    except Exception as e:
        print(f"Error processing multivariate events: {e}")
        return None

# Execute the function with default parameters
multivariate_events_data = get_multivariate_events()

#### Get Event

In [ ]:
"""
Cell 44: Get Event
Retrieves information about a specific event from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_event(event_ticker, with_nested_markets=False):
    """
    Fetches information about a specific event from Kalshi API

    Args:
        event_ticker: Event ticker
        with_nested_markets: Include nested markets in response (default False)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"

    # Build query parameters
    params = {}
    if with_nested_markets:
        params["with_nested_markets"] = with_nested_markets

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # The response structure depends on with_nested_markets parameter
        if with_nested_markets:
            event = data.get('event', {})
            markets = event.get('markets', []) if event else []
        else:
            event = data.get('event', {})
            markets = data.get('markets', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        if event:
            cursor.execute('''
                INSERT OR REPLACE INTO events (
                    event_ticker, series_ticker, sub_title, title,
                    collateral_return_type, mutually_exclusive, category,
                    strike_date, strike_period, available_on_brokers,
                    product_metadata
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                event.get('event_ticker'),
                event.get('series_ticker'),
                event.get('sub_title'),
                event.get('title'),
                event.get('collateral_return_type'),
                event.get('mutually_exclusive'),
                event.get('category'),
                event.get('strike_date'),
                event.get('strike_period'),
                event.get('available_on_brokers'),
                json.dumps(event.get('product_metadata', {}))
            ))

        # Store markets if present
        for market in markets:
            cursor.execute('''
                INSERT OR REPLACE INTO markets (
                    ticker, event_ticker, market_type, title, subtitle,
                    status, volume, last_price
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                market.get('ticker'),
                market.get('event_ticker'),
                market.get('market_type'),
                market.get('title'),
                market.get('subtitle'),
                market.get('status'),
                market.get('volume'),
                market.get('last_price')
            ))

        conn.commit()
        conn.close()

        print(f"Event Retrieved: {event.get('event_ticker')}")
        print(f"  Title: {event.get('title')}")
        print(f"  Category: {event.get('category')}")
        print(f"  Series: {event.get('series_ticker')}")
        print(f"  Strike Date: {event.get('strike_date')}")
        if markets:
            print(f"  Markets: {len(markets)} markets")
            for market in markets[:3]:
                print(f"    - {market.get('ticker')}: {market.get('title')}")
            if len(markets) > 3:
                print(f"    ... and {len(markets) - 3} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching event: {e}")
        return None
    except Exception as e:
        print(f"Error processing event: {e}")
        return None

# Example usage
# Replace with actual event ticker
event_ticker = "EXAMPLE-EVENT"

event_data = get_event(event_ticker, with_nested_markets=True)

#### Get Event Metadata

In [ ]:
"""
Cell 45: Get Event Metadata
Retrieves metadata for a specific event from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_event_metadata(event_ticker):
    """
    Fetches metadata for a specific event from Kalshi API

    Args:
        event_ticker: Event ticker
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}/metadata"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO event_metadata (
                event_ticker, image_url, settlement_sources,
                competition, competition_scope
            ) VALUES (?, ?, ?, ?, ?)
        ''', (
            event_ticker,
            data.get('image_url'),
            json.dumps(data.get('settlement_sources', [])),
            data.get('competition'),
            data.get('competition_scope')
        ))

        conn.commit()
        conn.close()

        print(f"Event Metadata Retrieved for {event_ticker}:")
        print(f"  Image URL: {data.get('image_url')}")
        print(f"  Competition: {data.get('competition')}")
        print(f"  Competition Scope: {data.get('competition_scope')}")

        settlement_sources = data.get('settlement_sources', [])
        if settlement_sources:
            print(f"  Settlement Sources: {len(settlement_sources)} sources")
            for source in settlement_sources[:3]:
                print(f"    - {source.get('name')}: {source.get('url')}")
            if len(settlement_sources) > 3:
                print(f"    ... and {len(settlement_sources) - 3} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching event metadata: {e}")
        return None
    except Exception as e:
        print(f"Error processing event metadata: {e}")
        return None

# Example usage
# Replace with actual event ticker
event_ticker = "EXAMPLE-EVENT"

event_metadata = get_event_metadata(event_ticker)

#### Get Event Forecast Percentile History

In [ ]:
"""
Cell 46: Get Event Forecast Percentile History
Retrieves forecast percentile history for a specific event (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_event_forecast_percentile_history(headers, series_ticker, event_ticker,
                                         percentiles, start_ts, end_ts, period_interval):
    """
    Fetches forecast percentile history for a specific event from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        series_ticker: The series ticker
        event_ticker: The event ticker
        percentiles: List of percentile values (0-10000, max 10 values)
        start_ts: Start timestamp for the range
        end_ts: End timestamp for the range
        period_interval: Period length (0 for 5-second, or 1, 60, 1440 minutes)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/series/{series_ticker}/events/{event_ticker}/forecast_percentile_history"

    # Build query parameters
    params = {
        "percentiles": percentiles,  # Array will be handled by requests
        "start_ts": start_ts,
        "end_ts": end_ts,
        "period_interval": period_interval
    }

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        forecast_history = data.get('forecast_history', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for forecast in forecast_history:
            event_ticker_data = forecast.get('event_ticker')
            end_period_ts = forecast.get('end_period_ts')
            period_interval_data = forecast.get('period_interval')
            percentile_points = forecast.get('percentile_points', [])

            for point in percentile_points:
                cursor.execute('''
                    INSERT INTO forecast_history (
                        event_ticker, series_ticker, end_period_ts, period_interval,
                        percentile, raw_numerical_forecast, numerical_forecast,
                        formatted_forecast
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    event_ticker_data,
                    series_ticker,
                    end_period_ts,
                    period_interval_data,
                    point.get('percentile'),
                    point.get('raw_numerical_forecast'),
                    point.get('numerical_forecast'),
                    point.get('formatted_forecast')
                ))

        conn.commit()
        conn.close()

        print(f"Forecast History Retrieved for {event_ticker}:")
        print(f"  Series: {series_ticker}")
        print(f"  Percentiles: {percentiles}")
        print(f"  Period Interval: {period_interval} {'seconds' if period_interval == 0 else 'minutes'}")
        print(f"  Data Points: {len(forecast_history)}")

        if forecast_history:
            first_point = forecast_history[0]
            if first_point.get('percentile_points'):
                print(f"  Sample Forecast: {first_point['percentile_points'][0].get('formatted_forecast')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching forecast history: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing forecast history: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    series_ticker = "EXAMPLE-SERIES"
    event_ticker = "EXAMPLE-EVENT"
    percentiles = [2500, 5000, 7500]  # 25th, 50th, 75th percentiles
    start_ts = 1698768000  # Example timestamp
    end_ts = 1698854400    # Example timestamp
    period_interval = 60   # 1 hour periods

    forecast_data = get_event_forecast_percentile_history(
        headers, series_ticker, event_ticker, percentiles,
        start_ts, end_ts, period_interval
    )
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    forecast_data = None

## **Live Data**

#### Get Live Data

In [ ]:
"""
Cell 47: Get Live Data
Retrieves live data for a specific milestone from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_live_data(data_type, milestone_id):
    """
    Fetches live data for a specific milestone from Kalshi API

    Args:
        data_type: Type of live data
        milestone_id: Milestone ID
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/live_data/{data_type}/milestone/{milestone_id}"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        live_data = data.get('live_data', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO live_data (type, milestone_id, details)
            VALUES (?, ?, ?)
        ''', (
            live_data.get('type'),
            milestone_id,
            json.dumps(live_data.get('details', {}))
        ))

        conn.commit()
        conn.close()

        print(f"Live Data Retrieved:")
        print(f"  Type: {live_data.get('type')}")
        print(f"  Milestone ID: {milestone_id}")
        details = live_data.get('details', {})
        if details:
            print(f"  Details: {len(details)} fields")
            for key in list(details.keys())[:5]:
                print(f"    - {key}: {details[key]}")
            if len(details) > 5:
                print(f"    ... and {len(details) - 5} more fields")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching live data: {e}")
        return None
    except Exception as e:
        print(f"Error processing live data: {e}")
        return None

# Example usage
# Replace with actual values
data_type = "election"  # Example type
milestone_id = "EXAMPLE-MILESTONE-ID"

live_data = get_live_data(data_type, milestone_id)

#### Get Multiple Live Data

In [ ]:
"""
Cell 48: Get Multiple Live Data
Retrieves live data for multiple milestones from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_multiple_live_data(milestone_ids):
    """
    Fetches live data for multiple milestones from Kalshi API

    Args:
        milestone_ids: List of milestone IDs (max 100)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/live_data/batch"

    # Build query parameters
    params = {
        "milestone_ids": milestone_ids  # Array will be handled by requests
    }

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        live_datas = data.get('live_datas', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for idx, live_data in enumerate(live_datas):
            # Use the corresponding milestone_id from the input
            milestone_id = milestone_ids[idx] if idx < len(milestone_ids) else f"milestone_{idx}"

            cursor.execute('''
                INSERT INTO live_data (type, milestone_id, details)
                VALUES (?, ?, ?)
            ''', (
                live_data.get('type'),
                milestone_id,
                json.dumps(live_data.get('details', {}))
            ))

        conn.commit()
        conn.close()

        print(f"Multiple Live Data Retrieved: {len(live_datas)} items")
        for idx, live_data in enumerate(live_datas[:5]):  # Show first 5
            milestone_id = milestone_ids[idx] if idx < len(milestone_ids) else f"milestone_{idx}"
            print(f"  - Milestone: {milestone_id}")
            print(f"    Type: {live_data.get('type')}")
            details = live_data.get('details', {})
            if details:
                print(f"    Details: {len(details)} fields")
        if len(live_datas) > 5:
            print(f"  ... and {len(live_datas) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching multiple live data: {e}")
        return None
    except Exception as e:
        print(f"Error processing multiple live data: {e}")
        return None

# Example usage
# Replace with actual milestone IDs
milestone_ids = [
    "MILESTONE-ID-1",
    "MILESTONE-ID-2",
    "MILESTONE-ID-3"
]

multiple_live_data = get_multiple_live_data(milestone_ids)

## **Incentive Programs**

#### Get Volume Incentives

In [ ]:
"""
Cell 49: Get Volume Incentives
Retrieves volume incentive programs from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime

def get_volume_incentives(status="all", incentive_type="all", limit=100, cursor=None):
    """
    Fetches volume incentive programs from Kalshi API

    Args:
        status: Status filter ('all', 'active', 'upcoming', 'closed', 'paid_out')
        incentive_type: Type filter ('all', 'liquidity', 'volume')
        limit: Number of results per page (1-10000, default 100)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/incentives"

    # Build query parameters
    params = {}
    if status:
        params["status"] = status
    if incentive_type:
        params["type"] = incentive_type
    if limit:
        params["limit"] = limit
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        incentive_programs = data.get('incentive_programs', [])
        next_cursor = data.get('next_cursor')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for program in incentive_programs:
            cursor_db.execute('''
                INSERT OR REPLACE INTO volume_incentives (
                    id, market_ticker, incentive_type, start_date, end_date,
                    period_reward, paid_out, discount_factor_bps, target_size
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                program.get('id'),
                program.get('market_ticker'),
                program.get('incentive_type'),
                program.get('start_date'),
                program.get('end_date'),
                program.get('period_reward'),
                program.get('paid_out'),
                program.get('discount_factor_bps'),
                program.get('target_size')
            ))

        conn.commit()
        conn.close()

        print(f"Volume Incentives Retrieved: {len(incentive_programs)} programs")
        print(f"  Status Filter: {status}")
        print(f"  Type Filter: {incentive_type}")

        for program in incentive_programs[:5]:  # Show first 5
            print(f"  - ID: {program.get('id')}")
            print(f"    Market: {program.get('market_ticker')}")
            print(f"    Type: {program.get('incentive_type')}")
            print(f"    Period Reward: {program.get('period_reward')}")
            print(f"    Paid Out: {program.get('paid_out')}")
        if len(incentive_programs) > 5:
            print(f"  ... and {len(incentive_programs) - 5} more")
        if next_cursor:
            print(f"  Next Cursor: {next_cursor}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching volume incentives: {e}")
        return None
    except Exception as e:
        print(f"Error processing volume incentives: {e}")
        return None

# Execute the function with default parameters
volume_incentives_data = get_volume_incentives()

## **FMC**

#### Get FCM Orders

In [ ]:
"""
Cell 51: Get FCM Orders
Retrieves FCM orders from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_fcm_orders(headers, subtrader_id, ticker=None, event_ticker=None,
                   min_ts=None, max_ts=None, status=None, limit=100, cursor=None):
    """
    Fetches FCM orders from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        subtrader_id: Restricts response to orders for specific subtrader (required)
        ticker: Filter by market ticker
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        min_ts: Filter orders after this Unix timestamp
        max_ts: Filter orders before this Unix timestamp
        status: Filter by status ('resting', 'canceled', 'executed')
        limit: Number of results per page (1-1000, default 100)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/fcm/orders"

    # Build query parameters
    params = {
        "subtrader_id": subtrader_id,
        "limit": limit
    }
    if ticker:
        params["ticker"] = ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if min_ts:
        params["min_ts"] = min_ts
    if max_ts:
        params["max_ts"] = max_ts
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        orders = data.get('orders', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for order in orders:
            cursor_db.execute('''
                INSERT OR REPLACE INTO fcm_orders (
                    order_id, user_id, subtrader_id, client_order_id, ticker,
                    side, action, type, status, yes_price, no_price,
                    yes_price_dollars, no_price_dollars, fill_count,
                    remaining_count, initial_count, taker_fees, maker_fees,
                    taker_fill_cost, maker_fill_cost, taker_fill_cost_dollars,
                    maker_fill_cost_dollars, queue_position, taker_fees_dollars,
                    maker_fees_dollars, expiration_time, created_time,
                    last_update_time, self_trade_prevention_type,
                    order_group_id, cancel_order_on_pause
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                order.get('order_id'),
                order.get('user_id'),
                subtrader_id,
                order.get('client_order_id'),
                order.get('ticker'),
                order.get('side'),
                order.get('action'),
                order.get('type'),
                order.get('status'),
                order.get('yes_price'),
                order.get('no_price'),
                order.get('yes_price_dollars'),
                order.get('no_price_dollars'),
                order.get('fill_count'),
                order.get('remaining_count'),
                order.get('initial_count'),
                order.get('taker_fees'),
                order.get('maker_fees'),
                order.get('taker_fill_cost'),
                order.get('maker_fill_cost'),
                order.get('taker_fill_cost_dollars'),
                order.get('maker_fill_cost_dollars'),
                order.get('queue_position'),
                order.get('taker_fees_dollars'),
                order.get('maker_fees_dollars'),
                order.get('expiration_time'),
                order.get('created_time'),
                order.get('last_update_time'),
                order.get('self_trade_prevention_type'),
                order.get('order_group_id'),
                order.get('cancel_order_on_pause')
            ))

        conn.commit()
        conn.close()

        print(f"FCM Orders Retrieved: {len(orders)} orders")
        print(f"  Subtrader ID: {subtrader_id}")
        for order in orders[:5]:  # Show first 5
            print(f"  - Order ID: {order.get('order_id')}")
            print(f"    Ticker: {order.get('ticker')}, Status: {order.get('status')}")
        if len(orders) > 5:
            print(f"  ... and {len(orders) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching FCM orders: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing FCM orders: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual subtrader ID
    subtrader_id = "your-subtrader-id"

    fcm_orders = get_fcm_orders(headers, subtrader_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    fcm_orders = None

#### Get FCM Positions

In [ ]:
"""
Cell 52: Get FCM Positions
Retrieves FCM positions from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_fcm_positions(headers, subtrader_id, ticker=None, event_ticker=None,
                      count_filter=None, settlement_status='unsettled',
                      limit=100, cursor=None):
    """
    Fetches FCM positions from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        subtrader_id: Restricts response to positions for specific subtrader (required)
        ticker: Filter by market ticker
        event_ticker: Filter by event ticker
        count_filter: Filter positions with non-zero values (comma-separated)
        settlement_status: 'all', 'unsettled', or 'settled' (default 'unsettled')
        limit: Number of results per page (1-1000, default 100)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/fcm/positions"

    # Build query parameters
    params = {
        "subtrader_id": subtrader_id,
        "settlement_status": settlement_status,
        "limit": limit
    }
    if ticker:
        params["ticker"] = ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if count_filter:
        params["count_filter"] = count_filter
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        market_positions = data.get('market_positions', [])
        event_positions = data.get('event_positions', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        # Store market positions
        for position in market_positions:
            cursor_db.execute('''
                INSERT INTO fcm_positions (
                    subtrader_id, ticker, position_type, total_traded,
                    total_traded_dollars, position, market_exposure,
                    market_exposure_dollars, realized_pnl, realized_pnl_dollars,
                    resting_orders_count, fees_paid, fees_paid_dollars,
                    last_updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                subtrader_id,
                position.get('ticker'),
                'market',
                position.get('total_traded'),
                position.get('total_traded_dollars'),
                position.get('position'),
                position.get('market_exposure'),
                position.get('market_exposure_dollars'),
                position.get('realized_pnl'),
                position.get('realized_pnl_dollars'),
                position.get('resting_orders_count'),
                position.get('fees_paid'),
                position.get('fees_paid_dollars'),
                position.get('last_updated_ts')
            ))

        # Store event positions
        for position in event_positions:
            cursor_db.execute('''
                INSERT INTO fcm_positions (
                    subtrader_id, event_ticker, position_type, total_cost,
                    total_cost_dollars, total_cost_shares, event_exposure,
                    event_exposure_dollars, realized_pnl, realized_pnl_dollars,
                    resting_order_count, fees_paid, fees_paid_dollars
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                subtrader_id,
                position.get('event_ticker'),
                'event',
                position.get('total_cost'),
                position.get('total_cost_dollars'),
                position.get('total_cost_shares'),
                position.get('event_exposure'),
                position.get('event_exposure_dollars'),
                position.get('realized_pnl'),
                position.get('realized_pnl_dollars'),
                position.get('resting_order_count'),
                position.get('fees_paid'),
                position.get('fees_paid_dollars')
            ))

        conn.commit()
        conn.close()

        print(f"FCM Positions Retrieved:")
        print(f"  Subtrader ID: {subtrader_id}")
        print(f"  Market Positions: {len(market_positions)}")
        print(f"  Event Positions: {len(event_positions)}")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching FCM positions: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing FCM positions: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual subtrader ID
    subtrader_id = "your-subtrader-id"

    fcm_positions = get_fcm_positions(headers, subtrader_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    fcm_positions = None

## **Structured Targets**

#### Get Structured Targets

In [ ]:
"""
Cell 53: Get Structured Targets
Retrieves structured targets from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_structured_targets(target_type=None, competition=None, page_size=100, cursor=None):
    """
    Fetches structured targets from Kalshi API

    Args:
        target_type: Filter by structured target type
        competition: Filter by competition
        page_size: Number of items per page (1-2000, default 100)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/structured_targets"

    # Build query parameters
    params = {"page_size": page_size}
    if target_type:
        params["type"] = target_type
    if competition:
        params["competition"] = competition
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        structured_targets = data.get('structured_targets', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for target in structured_targets:
            cursor_db.execute('''
                INSERT OR REPLACE INTO structured_targets (
                    id, name, type, details, source_id, last_updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                target.get('id'),
                target.get('name'),
                target.get('type'),
                json.dumps(target.get('details', {})),
                target.get('source_id'),
                target.get('last_updated_ts')
            ))

        conn.commit()
        conn.close()

        print(f"Structured Targets Retrieved: {len(structured_targets)} targets")
        for target in structured_targets[:5]:  # Show first 5
            print(f"  - ID: {target.get('id')}")
            print(f"    Name: {target.get('name')}")
            print(f"    Type: {target.get('type')}")
        if len(structured_targets) > 5:
            print(f"  ... and {len(structured_targets) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching structured targets: {e}")
        return None
    except Exception as e:
        print(f"Error processing structured targets: {e}")
        return None

# Execute the function
structured_targets_data = get_structured_targets()

#### Get Structured Target

In [ ]:
"""
Cell 54: Get Structured Target
Retrieves a specific structured target from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_structured_target(structured_target_id):
    """
    Fetches a specific structured target from Kalshi API

    Args:
        structured_target_id: Structured target ID
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/structured_targets/{structured_target_id}"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        target = data.get('structured_target', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO structured_targets (
                id, name, type, details, source_id, last_updated_ts
            ) VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            target.get('id'),
            target.get('name'),
            target.get('type'),
            json.dumps(target.get('details', {})),
            target.get('source_id'),
            target.get('last_updated_ts')
        ))

        conn.commit()
        conn.close()

        print(f"Structured Target Retrieved: {target.get('id')}")
        print(f"  Name: {target.get('name')}")
        print(f"  Type: {target.get('type')}")
        print(f"  Source ID: {target.get('source_id')}")
        print(f"  Last Updated: {target.get('last_updated_ts')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching structured target: {e}")
        return None
    except Exception as e:
        print(f"Error processing structured target: {e}")
        return None

# Example usage
# Replace with actual structured target ID
structured_target_id = "example-target-id"

structured_target = get_structured_target(structured_target_id)

## **Milestone**

#### Get Milestone

In [ ]:
"""
Cell 55: Get Milestone
Retrieves a specific milestone from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_milestone(milestone_id):
    """
    Fetches a specific milestone from Kalshi API

    Args:
        milestone_id: Milestone ID
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/milestones/{milestone_id}"

    try:
        # Make the API request (no headers needed)
        response = requests.get(url)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        milestone = data.get('milestone', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO milestones (
                id, category, type, start_date, end_date,
                related_event_tickers, title, notification_message,
                source_id, details, primary_event_tickers, last_updated_ts
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            milestone.get('id'),
            milestone.get('category'),
            milestone.get('type'),
            milestone.get('start_date'),
            milestone.get('end_date'),
            json.dumps(milestone.get('related_event_tickers', [])),
            milestone.get('title'),
            milestone.get('notification_message'),
            milestone.get('source_id'),
            json.dumps(milestone.get('details', {})),
            json.dumps(milestone.get('primary_event_tickers', [])),
            milestone.get('last_updated_ts')
        ))

        conn.commit()
        conn.close()

        print(f"Milestone Retrieved: {milestone.get('id')}")
        print(f"  Title: {milestone.get('title')}")
        print(f"  Category: {milestone.get('category')}")
        print(f"  Type: {milestone.get('type')}")
        print(f"  Start Date: {milestone.get('start_date')}")
        print(f"  End Date: {milestone.get('end_date')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching milestone: {e}")
        return None
    except Exception as e:
        print(f"Error processing milestone: {e}")
        return None

# Example usage
# Replace with actual milestone ID
milestone_id = "example-milestone-id"

milestone_data = get_milestone(milestone_id)

#### Get Milestones

In [ ]:
"""
Cell 56: Get Milestones
Retrieves list of milestones from the Kalshi exchange
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_milestones(limit, minimum_start_date=None, category=None, competition=None,
                   source_id=None, milestone_type=None, related_event_ticker=None,
                   cursor=None):
    """
    Fetches list of milestones from Kalshi API

    Args:
        limit: Number of milestones to return per page (1-500, required)
        minimum_start_date: Minimum start date filter (RFC3339 timestamp)
        category: Filter by milestone category
        competition: Filter by competition
        source_id: Filter by source id
        milestone_type: Filter by milestone type
        related_event_ticker: Filter by related event ticker
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/milestones"

    # Build query parameters
    params = {"limit": limit}
    if minimum_start_date:
        params["minimum_start_date"] = minimum_start_date
    if category:
        params["category"] = category
    if competition:
        params["competition"] = competition
    if source_id:
        params["source_id"] = source_id
    if milestone_type:
        params["type"] = milestone_type
    if related_event_ticker:
        params["related_event_ticker"] = related_event_ticker
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request (no headers needed)
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        milestones = data.get('milestones', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for milestone in milestones:
            cursor_db.execute('''
                INSERT OR REPLACE INTO milestones (
                    id, category, type, start_date, end_date,
                    related_event_tickers, title, notification_message,
                    source_id, details, primary_event_tickers, last_updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                milestone.get('id'),
                milestone.get('category'),
                milestone.get('type'),
                milestone.get('start_date'),
                milestone.get('end_date'),
                json.dumps(milestone.get('related_event_tickers', [])),
                milestone.get('title'),
                milestone.get('notification_message'),
                milestone.get('source_id'),
                json.dumps(milestone.get('details', {})),
                json.dumps(milestone.get('primary_event_tickers', [])),
                milestone.get('last_updated_ts')
            ))

        conn.commit()
        conn.close()

        print(f"Milestones Retrieved: {len(milestones)} milestones")
        for milestone in milestones[:5]:  # Show first 5
            print(f"  - ID: {milestone.get('id')}")
            print(f"    Title: {milestone.get('title')}")
            print(f"    Category: {milestone.get('category')}")
        if len(milestones) > 5:
            print(f"  ... and {len(milestones) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching milestones: {e}")
        return None
    except Exception as e:
        print(f"Error processing milestones: {e}")
        return None

# Example usage
# Fetch up to 100 milestones
milestones_data = get_milestones(limit=100)

## **Communications**

#### Get Communications ID

In [ ]:
"""
Cell 57: Get Communications ID
Retrieves the communications ID from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_communications_id(headers):
    """
    Fetches the communications ID from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/communications/id"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        communications_id = data.get('communications_id')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO communications_ids (communications_id)
            VALUES (?)
        ''', (communications_id,))

        conn.commit()
        conn.close()

        print(f"Communications ID Retrieved: {communications_id}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching communications ID: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing communications ID: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    comm_id = get_communications_id(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    comm_id = None

#### Get RFQs

In [ ]:
"""
Cell 58: Get RFQs
Retrieves Request for Quotes (RFQs) from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_rfqs(headers, cursor=None, limit=100, market_ticker=None, event_ticker=None,
            status=None, creator_user_id=None):
    """
    Fetches RFQs from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        cursor: Pagination cursor from previous response
        limit: Number of results per page (1-100, default 100)
        market_ticker: Filter by market ticker
        event_ticker: Filter by event ticker
        status: Filter by status
        creator_user_id: Filter by creator user ID
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/communications/rfqs"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if market_ticker:
        params["market_ticker"] = market_ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if status:
        params["status"] = status
    if creator_user_id:
        params["creator_user_id"] = creator_user_id

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        rfqs = data.get('rfqs', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for rfq in rfqs:
            cursor_db.execute('''
                INSERT OR REPLACE INTO rfqs (
                    id, creator_id, creator_user_id, market_ticker, contracts,
                    target_cost_centi_cents, status, created_ts, mve_collection_ticker,
                    mve_selected_legs, rest_remainder, cancellation_reason,
                    cancelled_ts, updated_ts
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                rfq.get('id'),
                rfq.get('creator_id'),
                rfq.get('creator_user_id'),
                rfq.get('market_ticker'),
                rfq.get('contracts'),
                rfq.get('target_cost_centi_cents'),
                rfq.get('status'),
                rfq.get('created_ts'),
                rfq.get('mve_collection_ticker'),
                json.dumps(rfq.get('mve_selected_legs', [])),
                rfq.get('rest_remainder'),
                rfq.get('cancellation_reason'),
                rfq.get('cancelled_ts'),
                rfq.get('updated_ts')
            ))

        conn.commit()
        conn.close()

        print(f"RFQs Retrieved: {len(rfqs)} RFQs")
        for rfq in rfqs[:5]:  # Show first 5
            print(f"  - ID: {rfq.get('id')}")
            print(f"    Market: {rfq.get('market_ticker')}")
            print(f"    Status: {rfq.get('status')}")
            print(f"    Contracts: {rfq.get('contracts')}")
        if len(rfqs) > 5:
            print(f"  ... and {len(rfqs) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching RFQs: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing RFQs: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    rfqs_data = get_rfqs(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    rfqs_data = None

#### Create RFQ

In [ ]:
"""
Cell 59: Create RFQ
Creates a new Request for Quote (RFQ) in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def create_rfq(headers, market_ticker, rest_remainder, contracts=None,
               target_cost_centi_cents=None, replace_existing=False, subtrader_id=None):
    """
    Creates a new RFQ in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        market_ticker: The ticker of the market for the RFQ (required)
        rest_remainder: Whether to rest remainder after execution (required)
        contracts: Number of contracts for the RFQ
        target_cost_centi_cents: Target cost in centi-cents
        replace_existing: Whether to delete existing RFQs (default False)
        subtrader_id: Subtrader ID for FCM members
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/communications/rfqs"

    # Build payload
    payload = {
        "market_ticker": market_ticker,
        "rest_remainder": rest_remainder,
        "replace_existing": replace_existing
    }
    if contracts is not None:
        payload["contracts"] = contracts
    if target_cost_centi_cents is not None:
        payload["target_cost_centi_cents"] = target_cost_centi_cents
    if subtrader_id:
        payload["subtrader_id"] = subtrader_id

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        rfq_id = data.get('id')

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO rfqs (
                id, market_ticker, contracts, target_cost_centi_cents,
                rest_remainder, status, created_ts
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            rfq_id,
            market_ticker,
            contracts,
            target_cost_centi_cents,
            rest_remainder,
            'open',  # New RFQs start as open
            datetime.utcnow().isoformat()
        ))

        conn.commit()
        conn.close()

        print(f"RFQ Created Successfully!")
        print(f"  RFQ ID: {rfq_id}")
        print(f"  Market: {market_ticker}")
        if contracts:
            print(f"  Contracts: {contracts}")
        if target_cost_centi_cents:
            print(f"  Target Cost: {target_cost_centi_cents} centi-cents")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating RFQ: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing RFQ creation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual values
    market_ticker = "EXAMPLE-MARKET"
    contracts = 100
    rest_remainder = True

    new_rfq = create_rfq(headers, market_ticker, rest_remainder, contracts=contracts)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_rfq = None

#### Get RFQ

In [ ]:
"""
Cell 60: Get RFQ
Retrieves a specific Request for Quote (RFQ) from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_rfq(headers, rfq_id):
    """
    Fetches a specific RFQ from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        rfq_id: RFQ ID
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/rfqs/{rfq_id}"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        rfq = data.get('rfq', {})

        # Store/update in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO rfqs (
                id, creator_id, creator_user_id, market_ticker, contracts,
                target_cost_centi_cents, status, created_ts, mve_collection_ticker,
                mve_selected_legs, rest_remainder, cancellation_reason,
                cancelled_ts, updated_ts
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            rfq.get('id'),
            rfq.get('creator_id'),
            rfq.get('creator_user_id'),
            rfq.get('market_ticker'),
            rfq.get('contracts'),
            rfq.get('target_cost_centi_cents'),
            rfq.get('status'),
            rfq.get('created_ts'),
            rfq.get('mve_collection_ticker'),
            json.dumps(rfq.get('mve_selected_legs', [])),
            rfq.get('rest_remainder'),
            rfq.get('cancellation_reason'),
            rfq.get('cancelled_ts'),
            rfq.get('updated_ts')
        ))

        conn.commit()
        conn.close()

        print(f"RFQ Retrieved: {rfq.get('id')}")
        print(f"  Market: {rfq.get('market_ticker')}")
        print(f"  Status: {rfq.get('status')}")
        print(f"  Contracts: {rfq.get('contracts')}")
        print(f"  Target Cost: {rfq.get('target_cost_centi_cents')} centi-cents")
        print(f"  Rest Remainder: {rfq.get('rest_remainder')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching RFQ: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing RFQ: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual RFQ ID
    rfq_id = "example-rfq-id"

    rfq_data = get_rfq(headers, rfq_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    rfq_data = None

#### Delete RFQ

In [ ]:
"""
Cell 61: Delete RFQ
Deletes a Request for Quote (RFQ) from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def delete_rfq(headers, rfq_id):
    """
    Deletes an RFQ from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        rfq_id: RFQ ID to delete
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/rfqs/{rfq_id}"

    try:
        # Make the API request with authentication headers
        response = requests.delete(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Update database (mark as deleted/cancelled)
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            UPDATE rfqs
            SET status = 'cancelled', cancelled_ts = ?
            WHERE id = ?
        ''', (datetime.utcnow().isoformat(), rfq_id))

        conn.commit()
        conn.close()

        print(f"RFQ Deleted Successfully!")
        print(f"  RFQ ID: {rfq_id}")

        return data if data else {"success": True, "deleted_id": rfq_id}

    except requests.exceptions.RequestException as e:
        print(f"Error deleting RFQ: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing RFQ deletion: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual RFQ ID to delete
    rfq_id = "example-rfq-id"

    deletion_result = delete_rfq(headers, rfq_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    deletion_result = None

#### Get Quotes

In [ ]:
"""
Cell 62: Get Quotes
Retrieves quotes from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_quotes(headers, cursor=None, limit=500, market_ticker=None, event_ticker=None,
               status=None, quote_creator_user_id=None, rfq_creator_user_id=None, rfq_id=None):
    """
    Fetches quotes from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        cursor: Pagination cursor from previous response
        limit: Number of results per page (1-500, default 500)
        market_ticker: Filter by market ticker
        event_ticker: Event ticker(s) to filter by (comma-separated, max 10)
        status: Filter by status
        quote_creator_user_id: Filter by quote creator user ID
        rfq_creator_user_id: Filter by RFQ creator user ID
        rfq_id: Filter by RFQ ID
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/communications/quotes"

    # Build query parameters
    params = {"limit": limit}
    if cursor:
        params["cursor"] = cursor
    if market_ticker:
        params["market_ticker"] = market_ticker
    if event_ticker:
        params["event_ticker"] = event_ticker
    if status:
        params["status"] = status
    if quote_creator_user_id:
        params["quote_creator_user_id"] = quote_creator_user_id
    if rfq_creator_user_id:
        params["rfq_creator_user_id"] = rfq_creator_user_id
    if rfq_id:
        params["rfq_id"] = rfq_id

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        quotes = data.get('quotes', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for quote in quotes:
            cursor_db.execute('''
                INSERT OR REPLACE INTO quotes (
                    id, rfq_id, creator_id, creator_user_id, rfq_creator_id,
                    rfq_creator_user_id, market_ticker, contracts, yes_bid,
                    no_bid, created_ts, updated_ts, status, accepted_side,
                    accepted_ts, confirmed_ts, executed_ts, cancelled_ts,
                    expired_ts, rest_remainder, cancellation_reason,
                    rfq_target_cost_centi_cents, rfq_creator_order_id,
                    creator_order_id
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                quote.get('id'),
                quote.get('rfq_id'),
                quote.get('creator_id'),
                quote.get('creator_user_id'),
                quote.get('rfq_creator_id'),
                quote.get('rfq_creator_user_id'),
                quote.get('market_ticker'),
                quote.get('contracts'),
                quote.get('yes_bid'),
                quote.get('no_bid'),
                quote.get('created_ts'),
                quote.get('updated_ts'),
                quote.get('status'),
                quote.get('accepted_side'),
                quote.get('accepted_ts'),
                quote.get('confirmed_ts'),
                quote.get('executed_ts'),
                quote.get('cancelled_ts'),
                quote.get('expired_ts'),
                quote.get('rest_remainder'),
                quote.get('cancellation_reason'),
                quote.get('rfq_target_cost_centi_cents'),
                quote.get('rfq_creator_order_id'),
                quote.get('creator_order_id')
            ))

        conn.commit()
        conn.close()

        print(f"Quotes Retrieved: {len(quotes)} quotes")
        for quote in quotes[:5]:  # Show first 5
            print(f"  - ID: {quote.get('id')}")
            print(f"    RFQ ID: {quote.get('rfq_id')}")
            print(f"    Market: {quote.get('market_ticker')}")
            print(f"    Status: {quote.get('status')}")
            print(f"    Yes Bid: {quote.get('yes_bid')}, No Bid: {quote.get('no_bid')}")
        if len(quotes) > 5:
            print(f"  ... and {len(quotes) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching quotes: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quotes: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    quotes_data = get_quotes(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    quotes_data = None

#### Create Quote

In [ ]:
"""
Cell 63: Create Quote
Creates a new quote for an RFQ in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def create_quote(headers, rfq_id, yes_bid, no_bid, rest_remainder):
    """
    Creates a new quote for an RFQ in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        rfq_id: The ID of the RFQ to quote on (required)
        yes_bid: The bid price for YES contracts in dollars (required)
        no_bid: The bid price for NO contracts in dollars (required)
        rest_remainder: Whether to rest remainder after execution (required)
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/communications/quotes"

    # Build payload
    payload = {
        "rfq_id": rfq_id,
        "yes_bid": yes_bid,
        "no_bid": no_bid,
        "rest_remainder": rest_remainder
    }

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        # Insert basic quote info (we'll get full details when we fetch the quote)
        cursor.execute('''
            INSERT INTO quotes (
                rfq_id, yes_bid, no_bid, rest_remainder,
                status, created_ts
            ) VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            rfq_id,
            yes_bid,
            no_bid,
            rest_remainder,
            'open',  # New quotes start as open
            datetime.utcnow().isoformat()
        ))

        conn.commit()
        conn.close()

        print(f"Quote Created Successfully!")
        print(f"  RFQ ID: {rfq_id}")
        print(f"  Yes Bid: ${yes_bid}")
        print(f"  No Bid: ${no_bid}")
        print(f"  Rest Remainder: {rest_remainder}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating quote: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quote creation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual values
    rfq_id = "example-rfq-id"
    yes_bid = "0.60"  # 60 cents
    no_bid = "0.40"   # 40 cents
    rest_remainder = True

    new_quote = create_quote(headers, rfq_id, yes_bid, no_bid, rest_remainder)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_quote = None

#### Get Quote

In [ ]:
"""
Cell 64: Get Quote
Retrieves a specific quote from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def get_quote(headers, quote_id):
    """
    Fetches a specific quote from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        quote_id: Quote ID
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/quotes/{quote_id}"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        quote = data.get('quote', {})

        # Store/update in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO quotes (
                id, rfq_id, creator_id, creator_user_id, rfq_creator_id,
                rfq_creator_user_id, market_ticker, contracts, yes_bid,
                no_bid, created_ts, updated_ts, status, accepted_side,
                accepted_ts, confirmed_ts, executed_ts, cancelled_ts,
                expired_ts, rest_remainder, cancellation_reason,
                rfq_target_cost_centi_cents, rfq_creator_order_id,
                creator_order_id
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            quote.get('id'),
            quote.get('rfq_id'),
            quote.get('creator_id'),
            quote.get('creator_user_id'),
            quote.get('rfq_creator_id'),
            quote.get('rfq_creator_user_id'),
            quote.get('market_ticker'),
            quote.get('contracts'),
            quote.get('yes_bid'),
            quote.get('no_bid'),
            quote.get('created_ts'),
            quote.get('updated_ts'),
            quote.get('status'),
            quote.get('accepted_side'),
            quote.get('accepted_ts'),
            quote.get('confirmed_ts'),
            quote.get('executed_ts'),
            quote.get('cancelled_ts'),
            quote.get('expired_ts'),
            quote.get('rest_remainder'),
            quote.get('cancellation_reason'),
            quote.get('rfq_target_cost_centi_cents'),
            quote.get('rfq_creator_order_id'),
            quote.get('creator_order_id')
        ))

        conn.commit()
        conn.close()

        print(f"Quote Retrieved: {quote.get('id')}")
        print(f"  RFQ ID: {quote.get('rfq_id')}")
        print(f"  Market: {quote.get('market_ticker')}")
        print(f"  Status: {quote.get('status')}")
        print(f"  Contracts: {quote.get('contracts')}")
        print(f"  Yes Bid: {quote.get('yes_bid')}, No Bid: {quote.get('no_bid')}")
        if quote.get('accepted_side'):
            print(f"  Accepted Side: {quote.get('accepted_side')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching quote: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quote: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual quote ID
    quote_id = "example-quote-id"

    quote_data = get_quote(headers, quote_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    quote_data = None

#### Delete Quote

In [ ]:
"""
Cell 65: Delete Quote
Deletes a quote from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def delete_quote(headers, quote_id):
    """
    Deletes a quote from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        quote_id: Quote ID to delete
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/quotes/{quote_id}"

    try:
        # Make the API request with authentication headers
        response = requests.delete(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Update database (mark as deleted/cancelled)
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            UPDATE quotes
            SET status = 'cancelled', cancelled_ts = ?
            WHERE id = ?
        ''', (datetime.utcnow().isoformat(), quote_id))

        conn.commit()
        conn.close()

        print(f"Quote Deleted Successfully!")
        print(f"  Quote ID: {quote_id}")

        return data if data else {"success": True, "deleted_id": quote_id}

    except requests.exceptions.RequestException as e:
        print(f"Error deleting quote: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quote deletion: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual quote ID to delete
    quote_id = "example-quote-id"

    deletion_result = delete_quote(headers, quote_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    deletion_result = None

#### Accept Quote

In [ ]:
"""
Cell 66: Accept Quote
Accepts a quote in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def accept_quote(headers, quote_id):
    """
    Accepts a quote in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        quote_id: Quote ID to accept
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/quotes/{quote_id}/accept"

    try:
        # Make the API request with authentication headers
        response = requests.put(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Update database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            UPDATE quotes
            SET status = 'accepted', accepted_ts = ?
            WHERE id = ?
        ''', (datetime.utcnow().isoformat(), quote_id))

        conn.commit()
        conn.close()

        print(f"Quote Accepted Successfully!")
        print(f"  Quote ID: {quote_id}")

        return data if data else {"success": True, "accepted_id": quote_id}

    except requests.exceptions.RequestException as e:
        print(f"Error accepting quote: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quote acceptance: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual quote ID to accept
    quote_id = "example-quote-id"

    accept_result = accept_quote(headers, quote_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    accept_result = None

#### Confirm Quote

In [ ]:
"""
Cell 67: Confirm Quote
Confirms a quote in the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime

def confirm_quote(headers, quote_id):
    """
    Confirms a quote in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        quote_id: Quote ID to confirm
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/communications/quotes/{quote_id}/confirm"

    try:
        # Make the API request with authentication headers
        response = requests.put(url, headers=headers)
        response.raise_for_status()

        # Check if there's any response data
        data = None
        if response.text:
            try:
                data = response.json()
            except:
                pass

        # Update database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            UPDATE quotes
            SET status = 'confirmed', confirmed_ts = ?
            WHERE id = ?
        ''', (datetime.utcnow().isoformat(), quote_id))

        conn.commit()
        conn.close()

        print(f"Quote Confirmed Successfully!")
        print(f"  Quote ID: {quote_id}")

        return data if data else {"success": True, "confirmed_id": quote_id}

    except requests.exceptions.RequestException as e:
        print(f"Error confirming quote: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing quote confirmation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual quote ID to confirm
    quote_id = "example-quote-id"

    confirm_result = confirm_quote(headers, quote_id)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    confirm_result = None

## **Collections**

#### Get Multivariate Event Collection

In [ ]:
"""
Cell 68: Get Multivariate Event Collection
Retrieves a specific multivariate event collection from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_multivariate_event_collection(headers, collection_ticker):
    """
    Fetches a specific multivariate event collection from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        collection_ticker: Collection ticker
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/multivariate_event_collections/{collection_ticker}"

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        collection = data.get('multivariate_contract', {})

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT OR REPLACE INTO multivariate_event_collections (
                collection_ticker, series_ticker, title, description,
                open_date, close_date, associated_events, associated_event_tickers,
                is_ordered, is_single_market_per_event, is_all_yes,
                size_min, size_max, functional_description
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            collection.get('collection_ticker'),
            collection.get('series_ticker'),
            collection.get('title'),
            collection.get('description'),
            collection.get('open_date'),
            collection.get('close_date'),
            json.dumps(collection.get('associated_events', [])),
            json.dumps(collection.get('associated_event_tickers', [])),
            collection.get('is_ordered'),
            collection.get('is_single_market_per_event'),
            collection.get('is_all_yes'),
            collection.get('size_min'),
            collection.get('size_max'),
            collection.get('functional_description')
        ))

        conn.commit()
        conn.close()

        print(f"Multivariate Event Collection Retrieved: {collection.get('collection_ticker')}")
        print(f"  Title: {collection.get('title')}")
        print(f"  Series: {collection.get('series_ticker')}")
        print(f"  Size Range: {collection.get('size_min')} - {collection.get('size_max')}")
        print(f"  Is Ordered: {collection.get('is_ordered')}")
        print(f"  Associated Events: {len(collection.get('associated_event_tickers', []))}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching multivariate event collection: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing multivariate event collection: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual collection ticker
    collection_ticker = "EXAMPLE-COLLECTION"

    collection_data = get_multivariate_event_collection(headers, collection_ticker)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    collection_data = None

#### Create Market In Multivariate Event Collection

In [ ]:
"""
Cell 69: Create Market In Multivariate Event Collection
Creates a market in a multivariate event collection (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def create_market_in_multivariate_collection(headers, collection_ticker, selected_markets):
    """
    Creates a market in a multivariate event collection in Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        collection_ticker: Collection ticker
        selected_markets: List of dicts with market_ticker, event_ticker, and side
                         Example: [{"market_ticker": "MARKET1", "event_ticker": "EVENT1", "side": "yes"}]
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/multivariate_event_collections/{collection_ticker}"

    # Build payload
    payload = {
        "selected_markets": selected_markets
    }

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        print(f"Market Created in Multivariate Collection!")
        print(f"  Collection: {collection_ticker}")
        print(f"  Event Ticker: {data.get('event_ticker')}")
        print(f"  Market Ticker: {data.get('market_ticker')}")
        print(f"  Selected Markets: {len(selected_markets)} markets")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error creating market in multivariate collection: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing market creation: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual values
    collection_ticker = "EXAMPLE-COLLECTION"
    selected_markets = [
        {
            "market_ticker": "MARKET1",
            "event_ticker": "EVENT1",
            "side": "yes"
        },
        {
            "market_ticker": "MARKET2",
            "event_ticker": "EVENT2",
            "side": "no"
        }
    ]

    new_market = create_market_in_multivariate_collection(headers, collection_ticker, selected_markets)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    new_market = None

#### Get Multivariate Event Collections

In [ ]:
"""
Cell 70: Get Multivariate Event Collections
Retrieves list of multivariate event collections from the Kalshi exchange (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_multivariate_event_collections(headers, status=None, associated_event_ticker=None,
                                       series_ticker=None, limit=None, cursor=None):
    """
    Fetches list of multivariate event collections from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        status: Filter by status ('unopened', 'open', 'closed')
        associated_event_ticker: Filter by associated event ticker
        series_ticker: Filter by series ticker
        limit: Maximum number of results (1-200)
        cursor: Pagination cursor from previous response
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/multivariate_event_collections"

    # Build query parameters
    params = {}
    if status:
        params["status"] = status
    if associated_event_ticker:
        params["associated_event_ticker"] = associated_event_ticker
    if series_ticker:
        params["series_ticker"] = series_ticker
    if limit:
        params["limit"] = limit
    if cursor:
        params["cursor"] = cursor

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        collections = data.get('multivariate_contracts', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor_db = conn.cursor()

        for collection in collections:
            cursor_db.execute('''
                INSERT OR REPLACE INTO multivariate_event_collections (
                    collection_ticker, series_ticker, title, description,
                    open_date, close_date, associated_events, associated_event_tickers,
                    is_ordered, is_single_market_per_event, is_all_yes,
                    size_min, size_max, functional_description
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                collection.get('collection_ticker'),
                collection.get('series_ticker'),
                collection.get('title'),
                collection.get('description'),
                collection.get('open_date'),
                collection.get('close_date'),
                json.dumps(collection.get('associated_events', [])),
                json.dumps(collection.get('associated_event_tickers', [])),
                collection.get('is_ordered'),
                collection.get('is_single_market_per_event'),
                collection.get('is_all_yes'),
                collection.get('size_min'),
                collection.get('size_max'),
                collection.get('functional_description')
            ))

        conn.commit()
        conn.close()

        print(f"Multivariate Event Collections Retrieved: {len(collections)} collections")
        for collection in collections[:5]:  # Show first 5
            print(f"  - {collection.get('collection_ticker')}: {collection.get('title')}")
            print(f"    Series: {collection.get('series_ticker')}")
        if len(collections) > 5:
            print(f"  ... and {len(collections) - 5} more")
        if data.get('cursor'):
            print(f"  Next Cursor: {data.get('cursor')}")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching multivariate event collections: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing multivariate event collections: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    collections_data = get_multivariate_event_collections(headers)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    collections_data = None

#### Get Multivariate Event Collection Lookup History

In [ ]:
"""
Cell 71: Get Multivariate Event Collection Lookup History
Retrieves lookup history for a multivariate event collection (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def get_multivariate_collection_lookup_history(headers, collection_ticker, lookback_seconds):
    """
    Fetches lookup history for a multivariate event collection from Kalshi API

    Args:
        headers: Dictionary containing authentication headers
        collection_ticker: Collection ticker
        lookback_seconds: Number of seconds to look back (10, 60, 300, or 3600)
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/multivariate_event_collections/{collection_ticker}/lookup"

    # Validate lookback_seconds
    valid_lookback = [10, 60, 300, 3600]
    if lookback_seconds not in valid_lookback:
        print(f"Error: lookback_seconds must be one of {valid_lookback}")
        return None

    # Build query parameters
    params = {
        "lookback_seconds": lookback_seconds
    }

    try:
        # Make the API request with authentication headers
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the response
        data = response.json()
        lookup_points = data.get('lookup_points', [])

        # Store in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        for point in lookup_points:
            cursor.execute('''
                INSERT INTO multivariate_lookups (
                    collection_ticker, event_ticker, market_ticker,
                    selected_markets, last_queried_ts
                ) VALUES (?, ?, ?, ?, ?)
            ''', (
                collection_ticker,
                point.get('event_ticker'),
                point.get('market_ticker'),
                json.dumps(point.get('selected_markets', [])),
                point.get('last_queried_ts')
            ))

        conn.commit()
        conn.close()

        print(f"Lookup History Retrieved for {collection_ticker}:")
        print(f"  Lookback Period: {lookback_seconds} seconds")
        print(f"  Lookup Points: {len(lookup_points)}")
        for point in lookup_points[:5]:  # Show first 5
            print(f"  - Event: {point.get('event_ticker')}, Market: {point.get('market_ticker')}")
            print(f"    Last Queried: {point.get('last_queried_ts')}")
        if len(lookup_points) > 5:
            print(f"  ... and {len(lookup_points) - 5} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching lookup history: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing lookup history: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual collection ticker
    collection_ticker = "EXAMPLE-COLLECTION"
    lookback_seconds = 300  # 5 minutes

    lookup_history = get_multivariate_collection_lookup_history(headers, collection_ticker, lookback_seconds)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    lookup_history = None

#### Lookup Tickers For Market In Multivariate Event Collection

In [ ]:
"""
Cell 72: Lookup Tickers For Market In Multivariate Event Collection
Looks up tickers for a market in a multivariate event collection (requires authentication)
"""

import requests
import sqlite3
from datetime import datetime
import json

def lookup_tickers_for_multivariate_market(headers, collection_ticker, selected_markets):
    """
    Looks up tickers for a market in a multivariate event collection

    Args:
        headers: Dictionary containing authentication headers
        collection_ticker: Collection ticker
        selected_markets: List of dicts with market_ticker, event_ticker, and side
                         Example: [{"market_ticker": "MARKET1", "event_ticker": "EVENT1", "side": "yes"}]
    """
    url = f"https://api.elections.kalshi.com/trade-api/v2/multivariate_event_collections/{collection_ticker}/lookup"

    # Build payload
    payload = {
        "selected_markets": selected_markets
    }

    # Add Content-Type to headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = "application/json"

    try:
        # Make the API request (PUT)
        response = requests.put(url, json=payload, headers=request_headers)
        response.raise_for_status()

        # Parse the response
        data = response.json()

        # Store lookup in database
        conn = sqlite3.connect('kalshi_data.db')
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO multivariate_lookups (
                collection_ticker, event_ticker, market_ticker,
                selected_markets, last_queried_ts
            ) VALUES (?, ?, ?, ?, ?)
        ''', (
            collection_ticker,
            data.get('event_ticker'),
            data.get('market_ticker'),
            json.dumps(selected_markets),
            datetime.utcnow().isoformat()
        ))

        conn.commit()
        conn.close()

        print(f"Ticker Lookup Completed!")
        print(f"  Collection: {collection_ticker}")
        print(f"  Event Ticker: {data.get('event_ticker')}")
        print(f"  Market Ticker: {data.get('market_ticker')}")
        print(f"  Selected Markets: {len(selected_markets)} markets")
        for market in selected_markets[:3]:
            print(f"    - {market.get('event_ticker')}/{market.get('market_ticker')} ({market.get('side')})")
        if len(selected_markets) > 3:
            print(f"    ... and {len(selected_markets) - 3} more")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error looking up tickers: {e}")
        if hasattr(e.response, 'text'):
            print(f"Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"Error processing ticker lookup: {e}")
        return None

# Example usage
# Note: headers variable should already be defined with authentication details
try:
    # Replace with actual values
    collection_ticker = "EXAMPLE-COLLECTION"
    selected_markets = [
        {
            "market_ticker": "MARKET1",
            "event_ticker": "EVENT1",
            "side": "yes"
        },
        {
            "market_ticker": "MARKET2",
            "event_ticker": "EVENT2",
            "side": "no"
        }
    ]

    lookup_result = lookup_tickers_for_multivariate_market(headers, collection_ticker, selected_markets)
except NameError:
    print("Headers not defined. Please ensure authentication headers are set before running this cell.")
    lookup_result = None

# **Next**